In [ ]:
!pip install -q pydantic
!pip install -q tqdm

In [2]:
import pandas as pd
import numpy as np
import requests
import json
import time
from dataclasses import dataclass
import pydantic
from tqdm import tqdm

In [3]:
total_tarefas = 50

# Cria a barra de progresso
with tqdm(total=total_tarefas, desc="Progresso", unit="tarefas") as pbar:
    for i in range(total_tarefas):
        time.sleep(0.1)  # Simula uma operação
        pbar.update(1)  # Atualiza a barra com cada tarefa concluída



Progresso: 100%|██████████| 50/50 [00:05<00:00,  9.17tarefas/s]


In [8]:
# Configurações específicas para cada sensor
sensores_config = {
    'Umidade_do_solo': {'Entrada_minima': 0, 'Entrada_maxima': 100, 'Entrada_um': '%Vol', 'Saida_minima': 0.3, 'Saida_maxima': 1, 'Saida_um': 'V', 'Ganho': 5},
    'Temperatura_do_solo': {'Entrada_minima': -35, 'Entrada_maxima': 50, 'Entrada_um': '°C', 'Saida_minima': 0, 'Saida_maxima': 5, 'Saida_um': 'V', 'Ganho': 2.5},
    'pH_do_solo': {'Entrada_minima': 0, 'Entrada_maxima': 14, 'Entrada_um': 'pH', 'Saida_minima': 0, 'Saida_maxima': 0.4, 'Saida_um': 'V', 'Ganho': 1.667},
    'Nivel_de_nutrientes': {'Entrada_minima': 0, 'Entrada_maxima': 100, 'Entrada_um': 'ppm', 'Saida_minima': 0, 'Saida_maxima': 5, 'Saida_um': 'V', 'Ganho': 1.250},
    'Radiacao_solar': {'Entrada_minima': 0, 'Entrada_maxima': 3000, 'Entrada_um': 'W/m²', 'Saida_minima': 0, 'Saida_maxima': 10, 'Saida_um': 'V', 'Ganho': 1},
    'Umidade_relativa_do_ar': {'Entrada_minima': 0, 'Entrada_maxima': 100, 'Entrada_um': '%', 'Saida_minima': 0, 'Saida_maxima': 3.3, 'Saida_um': 'V', 'Ganho': 0.833},
    'Temperatura_ambiente': {'Entrada_minima': -55, 'Entrada_maxima': 150, 'Entrada_um': '°C', 'Saida_minima': 0, 'Saida_maxima': 2.1, 'Saida_um': 'V', 'Ganho': 0.714}
    }

In [9]:
def sensores_config_initializer(sensores_config):
    data_sensores_config = {
    'sensor': list(sensores_config.keys()),
    'Entrada_minima': np.zeros(len(sensores_config.keys())).tolist(),
    'Entrada_maxima': np.zeros(len(sensores_config.keys())).tolist(),
    'Entrada_um': np.zeros(len(sensores_config.keys())).tolist(),
    'Saida_minima': np.zeros(len(sensores_config.keys())).tolist(),
    'Saida_maxima': np.zeros(len(sensores_config.keys())).tolist(),
    'Saida_um': np.zeros(len(sensores_config.keys())).tolist(),
    'K': np.zeros(len(sensores_config.keys())).tolist(),
    'K_um': np.zeros(len(sensores_config.keys())).tolist(),
    'a': np.zeros(len(sensores_config.keys())).tolist(),
    'a_um': np.zeros(len(sensores_config.keys())).tolist(),
    'Modelo_matematico': np.zeros(len(sensores_config.keys())).tolist(),
    'Ganho': np.zeros(len(sensores_config.keys())).tolist(),
    }
    df_sensores_config = pd.DataFrame(data_sensores_config)
    
    total_tarefas = len(list(sensores_config.keys()))

    # Cria a barra de progresso
    with tqdm(total=total_tarefas, desc="Sensores carregados", unit="sensores") as pbar:
        # Loop para preenchimento
        for sensor, config in sensores_config.items():
            print(sensor, config)
            df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'Entrada_minima'] = config['Entrada_minima']
            df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'Entrada_maxima'] = config['Entrada_maxima']
            df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'Entrada_um'] = config['Entrada_um']
            df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'Saida_minima'] = config['Saida_minima']
            df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'Saida_maxima'] = config['Saida_maxima']
            df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'Saida_um'] = config['Saida_um']
            
            # Cálculo dos valores de K e a
            K = (config['Saida_maxima'] - config['Saida_minima']) / (config['Entrada_maxima'] - config['Entrada_minima'])
            a = config['Saida_minima'] - (K * config['Entrada_minima'])
            
            df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'K']= K
            
            if '/' in config['Entrada_um']:
                num = config['Entrada_um'].split('/')[0]
                den = config['Entrada_um'].split('/')[1]
                k_um=f"{config['Saida_um']}. {den} /{num}"
                df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'K_um'] = k_um
                
            else:  
                k_um= f"{config['Saida_um']}/{config['Entrada_um']}"             
                df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'K_um'] = k_um
            
            df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'a'] = a
            df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'a_um'] = config['Saida_um']
            
            # Preenchimento do modelo matemático
            df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'Modelo_matematico'] = f"O(I)={np.round(K, 3)} {k_um}.I + {np.round(a, 3)} {config['Saida_um']}"
            df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'Ganho'] = config['Ganho']
            
            
            time.sleep(0.2)  # Simula uma operação
            pbar.update(1)  # Atualiza a barra com cada tarefa concluída
        
    return df_sensores_config

In [10]:
sensor_conf = sensores_config_initializer(sensores_config)
sensor_conf 

Sensores carregados:   0%|          | 0/7 [00:00<?, ?sensores/s]C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\3522803227.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '%Vol' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'Entrada_um'] = config['Entrada_um']
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\3522803227.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'Saida_um'] = config['Saida_um']
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\3522803227.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an er

Umidade_do_solo {'Entrada_minima': 0, 'Entrada_maxima': 100, 'Entrada_um': '%Vol', 'Saida_minima': 0.3, 'Saida_maxima': 1, 'Saida_um': 'V', 'Ganho': 5}


Sensores carregados:  14%|█▍        | 1/7 [00:00<00:01,  4.28sensores/s]

Temperatura_do_solo {'Entrada_minima': -35, 'Entrada_maxima': 50, 'Entrada_um': '°C', 'Saida_minima': 0, 'Saida_maxima': 5, 'Saida_um': 'V', 'Ganho': 2.5}


Sensores carregados:  29%|██▊       | 2/7 [00:00<00:01,  3.54sensores/s]

...

Umidade_relativa_do_ar {'Entrada_minima': 0, 'Entrada_maxima': 100, 'Entrada_um': '%', 'Saida_minima': 0, 'Saida_maxima': 3.3, 'Saida_um': 'V', 'Ganho': 0.833}


Sensores carregados:  86%|████████▌ | 6/7 [00:01<00:00,  3.71sensores/s]

Temperatura_ambiente {'Entrada_minima': -55, 'Entrada_maxima': 150, 'Entrada_um': '°C', 'Saida_minima': 0, 'Saida_maxima': 2.1, 'Saida_um': 'V', 'Ganho': 0.714}


Sensores carregados: 100%|██████████| 7/7 [00:01<00:00,  3.74sensores/s]


,sensor,Entrada_minima,Entrada_maxima,Entrada_um,Saida_minima,Saida_maxima,Saida_um,K,K_um,a,a_um,Modelo_matematico,Ganho
0,Umidade_do_solo,0.0,100.0,%Vol,0.3,1.0,V,0.007000,V/%Vol,0.300000,V,O(I)=0.007 V/%Vol.I + 0.3 V,5.000
1,Temperatura_do_solo,-35.0,50.0,°C,0.0,5.0,V,0.058824,V/°C,2.058824,V,O(I)=0.059 V/°C.I + 2.059 V,2.500
2,pH_do_solo,0.0,14.0,pH,0.0,0.4,V,0.028571,V/pH,0.000000,V,O(I)=0.029 V/pH.I + 0.0 V,1.667
3,Nivel_de_nutrientes,0.0,100.0,ppm,0.0,5.0,V,0.050000,V/ppm,0.000000,V,O(I)=0.05 V/ppm.I + 0.0 V,1.250
4,Radiacao_solar,0.0,3000.0,W/m²,0.0,10.0,V,0.003333,V. m² /W,0.000000,V,O(I)=0.003 V. m² /W.I + 0.0 V,1.000
5,Umidade_relativa_do_ar,0.0,100.0,%,0.0,3.3,V,0.033000,V/%,0.000000,V,O(I)=0.033 V/%.I + 0.0 V,0.833
6,Temperatura_ambiente,-55.0,150.0,°C,0.0,2.1,V,0.010244,V/°C,0.563415,V,O(I)=0.01 V/°C.I + 0.563 V,0.714


In [11]:
def add_sensor(sensor_config, df_sensores_config):
    # Adicionando novo sensor ao DataFrame
    
    sensor_name = list(sensor_config.keys())[0]
    sensores = list(df_sensores_config.columns)
    if sensor_name in sensores:
        return print(f"Sensor '{sensor_name}' já existe na lista.")
    
    else: 
        data_sensor_config = {
        'sensor': list(sensor_config.keys()),
        'Entrada_minima': np.zeros(len(sensor_config.keys())).tolist(),
        'Entrada_maxima': np.zeros(len(sensor_config.keys())).tolist(),
        'Entrada_um': np.zeros(len(sensor_config.keys())).tolist(),
        'Saida_minima': np.zeros(len(sensor_config.keys())).tolist(),
        'Saida_maxima': np.zeros(len(sensor_config.keys())).tolist(),
        'Saida_um': np.zeros(len(sensor_config.keys())).tolist(),
        'K': np.zeros(len(sensor_config.keys())).tolist(),
        'K_um': np.zeros(len(sensor_config.keys())).tolist(),
        'a': np.zeros(len(sensor_config.keys())).tolist(),
        'a_um': np.zeros(len(sensor_config.keys())).tolist(),
        'Modelo_matematico': np.zeros(len(sensor_config.keys())).tolist(),
        'Ganho': np.zeros(len(sensor_config.keys())).tolist(),
        }
        df_sensor_config = pd.DataFrame(data_sensor_config)
        
        total_tarefas = len(list(sensor_config.keys()))

        # Cria a barra de progresso
        with tqdm(total=total_tarefas, desc="Sensores carregados", unit="sensores") as pbar:
            # Loop para preenchimento
            for sensor, config in sensor_config.items():
                print(sensor, config)
                df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'Entrada_minima'] = config['Entrada_minima']
                df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'Entrada_maxima'] = config['Entrada_maxima']
                df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'Entrada_um'] = config['Entrada_um']
                df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'Saida_minima'] = config['Saida_minima']
                df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'Saida_maxima'] = config['Saida_maxima']
                df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'Saida_um'] = config['Saida_um']
                
                # Cálculo dos valores de K e a
                K = (config['Saida_maxima'] - config['Saida_minima']) / (config['Entrada_maxima'] - config['Entrada_minima'])
                a = config['Saida_minima'] - (K * config['Entrada_minima'])
                
                df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'K'] = K
                
                
                if '/' in config['Entrada_um']:
                   num = config['Entrada_um'].split('/')[0]
                   den = config['Entrada_um'].split('/')[1]
                   k_um = f"{config['Saida_um']}. {den} /{num}"
                   df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'K_um'] = f"{config['Saida_um']}. {den} /{num}"
                
                else:
                    k_um=  f"{config['Saida_um']}/{config['Entrada_um']}"               
                    df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'K_um'] = k_um
                
                df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'a'] = a
                df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'a_um'] = config['Saida_um']
                
                # Preenchimento do modelo matemático
                df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'Modelo_matematico'] = f"O(I)={np.round(K, 3)} {k_um}.I + {np.round(a, 3)} {config['Saida_um']}"
                df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'Ganho'] = config['Ganho']
                
                time.sleep(0.2)  # Simula uma operação
                pbar.update(1)  # Atualiza a barra com cada tarefa concluída
        

        
        # Adicionando a linha ao DataFrame
        df_sensores_config = pd.concat([df_sensores_config, df_sensor_config], ignore_index=True)

        return df_sensores_config

In [12]:
sensor_config= {'Vazao': {'Entrada_minima': 0, 'Entrada_maxima': 1200, 'Entrada_um': 'L/min', 'Saida_minima': 1, 'Saida_maxima': 5, 'Saida_um': 'V', 'Ganho': 0.625}}

In [13]:
sensor_conf2= add_sensor(sensor_config, sensor_conf)
sensor_conf2

Sensores carregados:   0%|          | 0/1 [00:00<?, ?sensores/s]

C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\385980112.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'L/min' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'Entrada_um'] = config['Entrada_um']
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\385980112.py:39: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'Saida_um'] = config['Saida_um']
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\385980112.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V. min /L' has dtype incompatibl

Vazao {'Entrada_minima': 0, 'Entrada_maxima': 1200, 'Entrada_um': 'L/min', 'Saida_minima': 1, 'Saida_maxima': 5, 'Saida_um': 'V', 'Ganho': 0.625}


Sensores carregados: 100%|██████████| 1/1 [00:00<00:00,  4.15sensores/s]


,sensor,Entrada_minima,Entrada_maxima,Entrada_um,Saida_minima,Saida_maxima,Saida_um,K,K_um,a,a_um,Modelo_matematico,Ganho
0,Umidade_do_solo,0.0,100.0,%Vol,0.3,1.0,V,0.007000,V/%Vol,0.300000,V,O(I)=0.007 V/%Vol.I + 0.3 V,5.000
1,Temperatura_do_solo,-35.0,50.0,°C,0.0,5.0,V,0.058824,V/°C,2.058824,V,O(I)=0.059 V/°C.I + 2.059 V,2.500
2,pH_do_solo,0.0,14.0,pH,0.0,0.4,V,0.028571,V/pH,0.000000,V,O(I)=0.029 V/pH.I + 0.0 V,1.667
3,Nivel_de_nutrientes,0.0,100.0,ppm,0.0,5.0,V,0.050000,V/ppm,0.000000,V,O(I)=0.05 V/ppm.I + 0.0 V,1.250
4,Radiacao_solar,0.0,3000.0,W/m²,0.0,10.0,V,0.003333,V. m² /W,0.000000,V,O(I)=0.003 V. m² /W.I + 0.0 V,1.000
5,Umidade_relativa_do_ar,0.0,100.0,%,0.0,3.3,V,0.033000,V/%,0.000000,V,O(I)=0.033 V/%.I + 0.0 V,0.833
6,Temperatura_ambiente,-55.0,150.0,°C,0.0,2.1,V,0.010244,V/°C,0.563415,V,O(I)=0.01 V/°C.I + 0.563 V,0.714
7,Vazao,0.0,1200.0,L/min,1.0,5.0,V,0.003333,V. min /L,1.000000,V,O(I)=0.003 V. min /L.I + 1.0 V,0.625


In [14]:
def dell_sensor(sensor_name,df_sensores_config):
    df_sensores_config = df_sensores_config[df_sensores_config['sensor'] != sensor_name].reset_index(drop=True)
    return df_sensores_config

In [15]:
sensor_conf3 = dell_sensor("Vazao", sensor_conf2)
sensor_conf3

,sensor,Entrada_minima,Entrada_maxima,Entrada_um,Saida_minima,Saida_maxima,Saida_um,K,K_um,a,a_um,Modelo_matematico,Ganho
0,Umidade_do_solo,0.0,100.0,%Vol,0.3,1.0,V,0.007000,V/%Vol,0.300000,V,O(I)=0.007 V/%Vol.I + 0.3 V,5.000
1,Temperatura_do_solo,-35.0,50.0,°C,0.0,5.0,V,0.058824,V/°C,2.058824,V,O(I)=0.059 V/°C.I + 2.059 V,2.500
2,pH_do_solo,0.0,14.0,pH,0.0,0.4,V,0.028571,V/pH,0.000000,V,O(I)=0.029 V/pH.I + 0.0 V,1.667
3,Nivel_de_nutrientes,0.0,100.0,ppm,0.0,5.0,V,0.050000,V/ppm,0.000000,V,O(I)=0.05 V/ppm.I + 0.0 V,1.250
4,Radiacao_solar,0.0,3000.0,W/m²,0.0,10.0,V,0.003333,V. m² /W,0.000000,V,O(I)=0.003 V. m² /W.I + 0.0 V,1.000
5,Umidade_relativa_do_ar,0.0,100.0,%,0.0,3.3,V,0.033000,V/%,0.000000,V,O(I)=0.033 V/%.I + 0.0 V,0.833
6,Temperatura_ambiente,-55.0,150.0,°C,0.0,2.1,V,0.010244,V/°C,0.563415,V,O(I)=0.01 V/°C.I + 0.563 V,0.714


In [18]:
def map_to_base_decimal(value, min_value, max_value, base_decimal=1023):
    """
    Mapeia um valor para o intervalo de 0 até base_decimal com base nos valores mínimo e máximo.

    Parameters:
    - value (float): O valor que será mapeado.
    - min_value (float): O valor mínimo esperado na escala original.
    - max_value (float): O valor máximo esperado na escala original.
    - base_decimal (int): O valor máximo na escala de destino (padrão é 1023).

    Returns:
    - int: O valor mapeado para o intervalo de 0 até base_decimal.
    """
    # Garantir que o valor esteja dentro do intervalo [min_value, max_value]
    if value < min_value:
        value = min_value
    elif value > max_value:
        value = max_value
    
    # Normalizar o valor para o intervalo de 0 a base_decimal
    normalized_value = (value - min_value) / (max_value - min_value)
    mapped_value = (normalized_value * base_decimal)
    
    return mapped_value


In [17]:
def get_sensor_data(sensor_data, df_sensores_config, BASE_DECIMAL=1023):
    data_saida = {'sensor':  list(sensor_data.keys()),
                    'Valor_lido':np.zeros(len(sensor_data.keys())).tolist(),
                    'Valor_lido_um':np.zeros(len(sensor_data.keys())).tolist(),
                    'Valor_enviado':np.zeros(len(sensor_data.keys())).tolist(),
                    'Valor_enviado_um':np.zeros(len(sensor_data.keys())).tolist(),
                    'Valor_condicionado':np.zeros(len(sensor_data.keys())).tolist(),
                    'Valor_condicionado_um':np.zeros(len(sensor_data.keys())).tolist(),
                    'Valor_decimal':np.zeros(len(sensor_data.keys())).tolist()    
                     }
    
    df_sensores_leitura = pd.DataFrame(data_saida)
   
    total_tarefas = len(list(sensor_data.keys()))

    # Cria a barra de progresso
    with tqdm(total=total_tarefas, desc="Sensores carregados", unit="sensores") as pbar:
        # Iterar sobre cada sensor e preencher os valores no DataFrame
        for sensor, valor_lido in sensor_data.items():
            # Definir o valor lido do sensor
            df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_lido'] = valor_lido

            
            # Definir o valor lido com a unidade de entrada
            df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_lido_um'] = \
                df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'Entrada_um'].values[0]

            # Calcular o valor enviado
            df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_enviado'] = (df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'K'].values[0] * valor_lido) + df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'a'].values[0]

            # Definir a unidade do valor enviado
            df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_enviado_um'] = \
                df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'Saida_um'].values[0]

            # Calcular o valor condicionado
            df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_condicionado'] = \
                df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'Ganho'].values[0] * \
                df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_enviado'].values[0]

            # Definir a unidade do valor condicionado
            df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_condicionado_um'] = \
                df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'Saida_um'].values[0]

            # Calcular o valor decimal
            min_value= df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'Saida_minima'].values[0]* df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'Ganho'].values[0]
            max_value= df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'Saida_maxima'].values[0]* df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'Ganho'].values[0]
            #valor_enviado = df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_enviado'].values[0]
            valor_enviado = df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_condicionado'].values[0] #/ df_sensores_config.loc[df_sensores_config['sensor'] == sensor, 'Ganho'].values[0]
            df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_decimal'] = map_to_base_decimal(valor_enviado, min_value, max_value, base_decimal=BASE_DECIMAL)
            
            time.sleep(0.2)  # Simula uma operação
            pbar.update(1)  # Atualiza a barra com cada tarefa concluída
        
    # Exibir o DataFrame resultante
    return df_sensores_leitura

In [19]:
sensor_conf2

,sensor,Entrada_minima,Entrada_maxima,Entrada_um,Saida_minima,Saida_maxima,Saida_um,K,K_um,a,a_um,Modelo_matematico,Ganho
0,Umidade_do_solo,0.0,100.0,%Vol,0.3,1.0,V,0.007000,V/%Vol,0.300000,V,O(I)=0.007 V/%Vol.I + 0.3 V,5.000
1,Temperatura_do_solo,-35.0,50.0,°C,0.0,5.0,V,0.058824,V/°C,2.058824,V,O(I)=0.059 V/°C.I + 2.059 V,2.500
2,pH_do_solo,0.0,14.0,pH,0.0,0.4,V,0.028571,V/pH,0.000000,V,O(I)=0.029 V/pH.I + 0.0 V,1.667
3,Nivel_de_nutrientes,0.0,100.0,ppm,0.0,5.0,V,0.050000,V/ppm,0.000000,V,O(I)=0.05 V/ppm.I + 0.0 V,1.250
4,Radiacao_solar,0.0,3000.0,W/m²,0.0,10.0,V,0.003333,V. m² /W,0.000000,V,O(I)=0.003 V. m² /W.I + 0.0 V,1.000
5,Umidade_relativa_do_ar,0.0,100.0,%,0.0,3.3,V,0.033000,V/%,0.000000,V,O(I)=0.033 V/%.I + 0.0 V,0.833
6,Temperatura_ambiente,-55.0,150.0,°C,0.0,2.1,V,0.010244,V/°C,0.563415,V,O(I)=0.01 V/°C.I + 0.563 V,0.714
7,Vazao,0.0,1200.0,L/min,1.0,5.0,V,0.003333,V. min /L,1.000000,V,O(I)=0.003 V. min /L.I + 1.0 V,0.625


In [20]:
sensor_conf2['Entrada_minima']

0     0.0
1   -35.0
2     0.0
3     0.0
4     0.0
5     0.0
6   -55.0
7     0.0
Name: Entrada_minima, dtype: float64

In [21]:
# Lista de valores lidos para cada sensor (substitua pelos valores corretos)
sensor_data= {
    'Umidade_do_solo': 0,
    'Temperatura_do_solo': -35,
    'pH_do_solo': 0,
    'Nivel_de_nutrientes': 0,
    'Radiacao_solar': 0,
    'Umidade_relativa_do_ar': 0,
    'Temperatura_ambiente': -55,
    'Vazao': 0
}

In [22]:
df_sensores_leitura_min= get_sensor_data(sensor_data, sensor_conf2, BASE_DECIMAL=1023)
df_sensores_leitura_min

Sensores carregados:   0%|          | 0/8 [00:00<?, ?sensores/s]C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\2989346303.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '%Vol' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_lido_um'] = \
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\2989346303.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_enviado_um'] = \
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\2989346303.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future versi

,sensor,Valor_lido,Valor_lido_um,Valor_enviado,Valor_enviado_um,Valor_condicionado,Valor_condicionado_um,Valor_decimal
0,Umidade_do_solo,0.0,%Vol,0.3,V,1.500,V,0.0
1,Temperatura_do_solo,-35.0,°C,0.0,V,0.000,V,0.0
2,pH_do_solo,0.0,pH,0.0,V,0.000,V,0.0
3,Nivel_de_nutrientes,0.0,ppm,0.0,V,0.000,V,0.0
4,Radiacao_solar,0.0,W/m²,0.0,V,0.000,V,0.0
5,Umidade_relativa_do_ar,0.0,%,0.0,V,0.000,V,0.0
6,Temperatura_ambiente,-55.0,°C,0.0,V,0.000,V,0.0
7,Vazao,0.0,L/min,1.0,V,0.625,V,0.0


In [23]:
sensor_conf2['Entrada_maxima']

0     100.0
1      50.0
2      14.0
3     100.0
4    3000.0
5     100.0
6     150.0
7    1200.0
Name: Entrada_maxima, dtype: float64

In [24]:
# Lista de valores lidos para cada sensor (substitua pelos valores corretos)
sensor_data= {
    'Umidade_do_solo': 100,
    'Temperatura_do_solo': 50,
    'pH_do_solo': 14,
    'Nivel_de_nutrientes': 100,
    'Radiacao_solar': 3000,
    'Umidade_relativa_do_ar': 100,
    'Temperatura_ambiente': 150,
    'Vazao': 1200
}

In [25]:
df_sensores_leitura_max= get_sensor_data(sensor_data, sensor_conf2, BASE_DECIMAL=1023)
df_sensores_leitura_max

Sensores carregados:   0%|          | 0/8 [00:00<?, ?sensores/s]

C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\2989346303.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '%Vol' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_lido_um'] = \
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\2989346303.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_enviado_um'] = \
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\2989346303.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, ple

,sensor,Valor_lido,Valor_lido_um,Valor_enviado,Valor_enviado_um,Valor_condicionado,Valor_condicionado_um,Valor_decimal
0,Umidade_do_solo,100.0,%Vol,1.0,V,5.0000,V,1023.0
1,Temperatura_do_solo,50.0,°C,5.0,V,12.5000,V,1023.0
2,pH_do_solo,14.0,pH,0.4,V,0.6668,V,1023.0
3,Nivel_de_nutrientes,100.0,ppm,5.0,V,6.2500,V,1023.0
4,Radiacao_solar,3000.0,W/m²,10.0,V,10.0000,V,1023.0
5,Umidade_relativa_do_ar,100.0,%,3.3,V,2.7489,V,1023.0
6,Temperatura_ambiente,150.0,°C,2.1,V,1.4994,V,1023.0
7,Vazao,1200.0,L/min,5.0,V,3.1250,V,1023.0


In [26]:
sensor_conf2['Saida_maxima']

0     1.0
1     5.0
2     0.4
3     5.0
4    10.0
5     3.3
6     2.1
7     5.0
Name: Saida_maxima, dtype: float64

In [153]:
sensor_conf2

,sensor,Entrada_minima,Entrada_maxima,Entrada_um,Saida_minima,Saida_maxima,Saida_um,K,K_um,a,a_um,Modelo_matematico,Ganho
0,Umidade_do_solo,0.0,100.0,%Vol,0.3,1.0,V,0.007000,V/%Vol,0.300000,V,O(I)=0.007 V/%Vol.I + 0.3 V,5.000
1,Temperatura_do_solo,-35.0,50.0,°C,0.0,5.0,V,0.058824,V/°C,2.058824,V,O(I)=0.059 V/°C.I + 2.059 V,2.500
2,pH_do_solo,0.0,14.0,pH,0.0,0.4,V,0.028571,V/pH,0.000000,V,O(I)=0.029 V/pH.I + 0.0 V,1.667
3,Nivel_de_nutrientes,0.0,100.0,ppm,0.0,5.0,V,0.050000,V/ppm,0.000000,V,O(I)=0.05 V/ppm.I + 0.0 V,1.250
4,Radiacao_solar,0.0,3000.0,W/m²,0.0,10.0,V,0.003333,V. m² /W,0.000000,V,O(I)=0.003 V. m² /W.I + 0.0 V,1.000
5,Umidade_relativa_do_ar,0.0,100.0,%,0.0,3.3,V,0.033000,V/%,0.000000,V,O(I)=0.033 V/%.I + 0.0 V,0.833
6,Temperatura_ambiente,-55.0,150.0,°C,0.0,2.1,V,0.010244,V/°C,0.563415,V,O(I)=0.01 V/°C.I + 0.563 V,0.714
7,Vazao,0.0,1200.0,L/min,1.0,5.0,V,0.003333,V. min /L,1.000000,V,O(I)=0.003 V. min /L.I + 1.0 V,0.625


In [156]:
def sensor_min_max_test(df_sensor_config, min= True):
    
    # if self.df_sensores_config is None:
    #     return print('Por favor carrege o Json de inicialização dos sensores!')
    
    # else: 
 

        # if data_sensor_max is None:
        #     data_sensor_max = {}
        #     data_sensor_min= {}
        #     for s, vmin, vmax in zip(df_sensor_config['sensor'].values, df_sensor_config['Entrada_minima'].values, df_sensor_config['Entrada_maxima'].values )  
        #         data_sensor_min[s]= vmin
        #         data_sensor_max[s]= vmax

    data_sensor_max = {}
    data_sensor_min= {}
    for s, vmin, vmax in zip(df_sensor_config['sensor'].values, df_sensor_config['Entrada_minima'].values, df_sensor_config['Entrada_maxima'].values ):  
        data_sensor_min[s]= vmin
        data_sensor_max[s]= vmax

    if min == True:
        df_sensores_leitura_min = get_sensor_data(data_sensor_min, df_sensor_config, BASE_DECIMAL=1023)
        return df_sensores_leitura_min
    
    if min==False: 
        df_sensores_leitura_max= get_sensor_data(data_sensor_max, df_sensor_config, BASE_DECIMAL=1023)
        return df_sensores_leitura_max
        

In [157]:
sensor_min_max_test(sensor_conf2)

Sensores carregados:   0%|          | 0/8 [00:00<?, ?sensores/s]C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\2989346303.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '%Vol' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_lido_um'] = \
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\2989346303.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_enviado_um'] = \
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\2989346303.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future versi

,sensor,Valor_lido,Valor_lido_um,Valor_enviado,Valor_enviado_um,Valor_condicionado,Valor_condicionado_um,Valor_decimal
0,Umidade_do_solo,0.0,%Vol,0.3,V,1.500,V,0.0
1,Temperatura_do_solo,-35.0,°C,0.0,V,0.000,V,0.0
2,pH_do_solo,0.0,pH,0.0,V,0.000,V,0.0
3,Nivel_de_nutrientes,0.0,ppm,0.0,V,0.000,V,0.0
4,Radiacao_solar,0.0,W/m²,0.0,V,0.000,V,0.0
5,Umidade_relativa_do_ar,0.0,%,0.0,V,0.000,V,0.0
6,Temperatura_ambiente,-55.0,°C,0.0,V,0.000,V,0.0
7,Vazao,0.0,L/min,1.0,V,0.625,V,0.0


In [158]:
sensor_min_max_test(sensor_conf2, min=False)

Sensores carregados:   0%|          | 0/8 [00:00<?, ?sensores/s]C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\2989346303.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '%Vol' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_lido_um'] = \
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\2989346303.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_enviado_um'] = \
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\2989346303.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future versi

,sensor,Valor_lido,Valor_lido_um,Valor_enviado,Valor_enviado_um,Valor_condicionado,Valor_condicionado_um,Valor_decimal
0,Umidade_do_solo,100.0,%Vol,1.0,V,5.0000,V,1023.0
1,Temperatura_do_solo,50.0,°C,5.0,V,12.5000,V,1023.0
2,pH_do_solo,14.0,pH,0.4,V,0.6668,V,1023.0
3,Nivel_de_nutrientes,100.0,ppm,5.0,V,6.2500,V,1023.0
4,Radiacao_solar,3000.0,W/m²,10.0,V,10.0000,V,1023.0
5,Umidade_relativa_do_ar,100.0,%,3.3,V,2.7489,V,1023.0
6,Temperatura_ambiente,150.0,°C,2.1,V,1.4994,V,1023.0
7,Vazao,1200.0,L/min,5.0,V,3.1250,V,1023.0


In [28]:
def prepare_sensor_data(df_sensores_config, df_sensores_leitura, BASE_DECIMAL):
    json_data= {}
    conf_dict = df_sensores_config.to_dict('records')
    data_senores= df_sensores_leitura.to_dict('records')
    json_data["data_sensores"]= data_senores
    json_data["data_config"]= conf_dict
    json_data["BASE_SENSOR_DIGITAL"]= BASE_DECIMAL
    
    return json_data


In [29]:
json_data= prepare_sensor_data(sensor_conf2, df_sensores_leitura_max , BASE_DECIMAL=1023)
json_data

{'data_sensores': [{'sensor': 'Umidade_do_solo',
   'Valor_lido': 100.0,
   'Valor_lido_um': '%Vol',
   'Valor_enviado': 1.0,
   'Valor_enviado_um': 'V',
   'Valor_condicionado': 5.0,
   'Valor_condicionado_um': 'V',
   'Valor_decimal': 1023.0},
  {'sensor': 'Temperatura_do_solo',
   'Valor_lido': 50.0,
   'Valor_lido_um': '°C',
   'Valor_enviado': 5.0,
   'Valor_enviado_um': 'V',
   'Valor_condicionado': 12.5,
   'Valor_condicionado_um': 'V',
   'Valor_decimal': 1023.0},
  {'sensor': 'pH_do_solo',
   'Valor_lido': 14.0,
   'Valor_lido_um': 'pH',
   'Valor_enviado': 0.4,
   'Valor_enviado_um': 'V',
   'Valor_condicionado': 0.6668000000000001,
   'Valor_condicionado_um': 'V',
   'Valor_decimal': 1023.0},
  {'sensor': 'Nivel_de_nutrientes',
   'Valor_lido': 100.0,
   'Valor_lido_um': 'ppm',
   'Valor_enviado': 5.0,
   'Valor_enviado_um': 'V',
   'Valor_condicionado': 6.25,
   'Valor_condicionado_um': 'V',
   'Valor_decimal': 1023.0},
  {'sensor': 'Radiacao_solar',
   'Valor_lido': 3000.0

In [31]:
def mapValue(value, in_min, in_max, out_min, out_max):
  return np.round(((((value - in_min) * (out_max - out_min)) / (in_max - in_min)) + out_min),1)


In [30]:
def show_sensor_response(json_data):
    total_tarefas = len(json_data['data_config'])
    saida = "Saída: \n\n"
    # Cria a barra de progresso
    with tqdm(total=total_tarefas, desc="Sensores carregados", unit="sensores") as pbar:
        # Loop para preenchimento
        for i in range(len(json_data['data_config'])):
            data_config = json_data['data_config'][i]
            data_sensores = json_data['data_sensores'][i]
            sensor = data_config.get('sensor')
            val_in_min = data_config.get('Entrada_minima')
            val_in_max = data_config.get('Entrada_maxima')
            val_out_min = 0
            val_out_max = json_data['BASE_SENSOR_DIGITAL']
            um = data_config.get('Entrada_um')
            valor_decimal= data_sensores.get('Valor_decimal')
            valor_saida = mapValue(valor_decimal, val_out_min, val_out_max, val_in_min, val_in_max)
            #valor_saida = np.round(valor_saida,2)
            time.sleep(0.2)  # Simula uma operação
            pbar.update(1)  # Atualiza a barra com cada tarefa concluída
            
            saida = saida + f'{sensor}: {valor_saida} {um} \n'
    print(' ')
    print(saida)
        

In [32]:
show_sensor_response(json_data)

Sensores carregados: 100%|██████████| 8/8 [00:01<00:00,  4.93sensores/s]

 
Saída: 

Umidade_do_solo: 100.0 %Vol 
Temperatura_do_solo: 50.0 °C 
pH_do_solo: 14.0 pH 
Nivel_de_nutrientes: 100.0 ppm 
Radiacao_solar: 3000.0 W/m² 
Umidade_relativa_do_ar: 100.0 % 
Temperatura_ambiente: 150.0 °C 
Vazao: 1200.0 L/min 



In [ ]:
# URL do Node-RED
url = "http://127.0.0.1:1880/*"

In [34]:
def send_data_to_NodeRed(json_data, url):
    # Enviar os dados
    response = requests.post(url, json=json_data)

    # Verificar a resposta
    if response.status_code == 200:
        print("Dados enviados com sucesso!")
    else:
        print(f"Erro ao enviar dados: {response.status_code}")

In [36]:
send_data_to_NodeRed(json_data, url)

Dados enviados com sucesso!


In [205]:
umidade_do_solo= [0, 10, 20, 30, 40, 50, 60, 70, 80 , 90, 100]
temperatura_do_solo= [-35, -15, -10, 0 , 10, 15 , 20, 30, 35, 40, 50]
ph_do_solo=[0, 1, 2, 3, 6, 7, 8, 9, 10, 12, 14]
nivel_de_nutrientes= [0, 10, 20, 30, 40, 50, 60, 70, 80 , 90, 100]
radiacao_solar = [0, 100, 200, 300, 400, 500, 600, 700, 800 , 900, 3000]
umidade_relativa_do_ar= [0, 10, 20, 30, 40, 50, 60, 70, 80 , 90, 100]
temperatura_ambiente=[-55,-25, 0, 5, 15, 25, 50, 75, 100, 125,  150]
vazao= [0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 1200]

print(len(umidade_do_solo))
print(len(temperatura_do_solo))
print(len(ph_do_solo))
print(len(nivel_de_nutrientes))
print(len(radiacao_solar))
print(len(umidade_relativa_do_ar))
print(len(temperatura_ambiente))
print(len(vazao))

11
11
11
11
11
11
11
11


In [206]:
total_tarefas= len(vazao)
# Cria a barra de progresso
print("Iniciando...")
time.sleep(2)  # Pausa a execução por 2 segundos
with tqdm(total=total_tarefas, desc="Sensores carregados", unit="sensores") as pbar:
  # Loop para preenchimento
  for us, ts, ph, nn, rs,ur, ta, va in zip(umidade_do_solo, temperatura_do_solo, ph_do_solo, nivel_de_nutrientes, radiacao_solar,umidade_relativa_do_ar,temperatura_ambiente,vazao ):
    sensor_data= {
                    'Umidade_do_solo': us,
                    'Temperatura_do_solo': ts,
                    'pH_do_solo': ph,
                    'Nivel_de_nutrientes': nn,
                    'Radiacao_solar': rs,
                    'Umidade_relativa_do_ar': ur,
                    'Temperatura_ambiente': ta,
                    'Vazao': va
                  }
    
    df_sensores_leitura= get_sensor_data(sensor_data, sensor_conf2, BASE_DECIMAL=1023)
    json_data=prepare_sensor_data(sensor_conf2, df_sensores_leitura, BASE_DECIMAL=1023)
    # send_data_to_NodeRed(json_data, url)
    show_sensor_response(json_data)
    
    time.sleep(0.2)  # Simula uma operação
    pbar.update(1)  # Atualiza a barra com cada tarefa concluída

Iniciando...


Sensores carregados:   0%|          | 0/11 [00:00<?, ?sensores/s]C:\Users\Pichau\AppData\Local\Temp\ipykernel_14208\2989346303.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '%Vol' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_lido_um'] = \
C:\Users\Pichau\AppData\Local\Temp\ipykernel_14208\2989346303.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_enviado_um'] = \
C:\Users\Pichau\AppData\Local\Temp\ipykernel_14208\2989346303.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future v

 
Saída: 

Umidade_do_solo: 0.0 %Vol 
Temperatura_do_solo: -35.0 °C 
pH_do_solo: 0.0 pH 
Nivel_de_nutrientes: 0.0 ppm 
Radiacao_solar: 0.0 W/m² 
Umidade_relativa_do_ar: 0.0 % 
Temperatura_ambiente: -55.0 °C 
Vazao: 0.0 L/min 



C:\Users\Pichau\AppData\Local\Temp\ipykernel_14208\2989346303.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '%Vol' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_lido_um'] = \
C:\Users\Pichau\AppData\Local\Temp\ipykernel_14208\2989346303.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_enviado_um'] = \
C:\Users\Pichau\AppData\Local\Temp\ipykernel_14208\2989346303.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, 

 
Saída: 

Umidade_do_solo: 10.0 %Vol 
Temperatura_do_solo: -15.0 °C 
pH_do_solo: 1.0 pH 
Nivel_de_nutrientes: 10.0 ppm 
Radiacao_solar: 100.0 W/m² 
Umidade_relativa_do_ar: 10.0 % 
Temperatura_ambiente: -25.0 °C 
Vazao: 20.0 L/min 



...

C:\Users\Pichau\AppData\Local\Temp\ipykernel_14208\2989346303.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '%Vol' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_lido_um'] = \
C:\Users\Pichau\AppData\Local\Temp\ipykernel_14208\2989346303.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_enviado_um'] = \
C:\Users\Pichau\AppData\Local\Temp\ipykernel_14208\2989346303.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, 

 
Saída: 

Umidade_do_solo: 90.0 %Vol 
Temperatura_do_solo: 40.0 °C 
pH_do_solo: 12.0 pH 
Nivel_de_nutrientes: 90.0 ppm 
Radiacao_solar: 900.0 W/m² 
Umidade_relativa_do_ar: 90.0 % 
Temperatura_ambiente: 125.0 °C 
Vazao: 180.0 L/min 



C:\Users\Pichau\AppData\Local\Temp\ipykernel_14208\2989346303.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '%Vol' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_lido_um'] = \
C:\Users\Pichau\AppData\Local\Temp\ipykernel_14208\2989346303.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensores_leitura.loc[df_sensores_leitura['sensor'] == sensor, 'Valor_enviado_um'] = \
C:\Users\Pichau\AppData\Local\Temp\ipykernel_14208\2989346303.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, 

 
Saída: 

Umidade_do_solo: 100.0 %Vol 
Temperatura_do_solo: 50.0 °C 
pH_do_solo: 14.0 pH 
Nivel_de_nutrientes: 100.0 ppm 
Radiacao_solar: 3000.0 W/m² 
Umidade_relativa_do_ar: 100.0 % 
Temperatura_ambiente: 150.0 °C 
Vazao: 1200.0 L/min 



Sensores carregados: 100%|██████████| 11/11 [00:38<00:00,  3.46s/sensores]


**Código do Node Red em Java Script**

In [ ]:
let umidadeSolo, temperaturaSolo, pHSolo, nivelNutrientes, radiacaoSolar, umidadeRelativa, temperaturaAmbiente, vazao;

for (let i = 0; i < dataConfig.length; i++) {
    let sensorConfig = dataConfig[i];
    let sensorData = dataSensores[i];
    let sensor = sensorConfig.sensor;
    let valorAD = sensorData.Valor_decimal;
    let inMin = sensorConfig.Entrada_minima;
    let inMax = sensorConfig.Entrada_maxima;
    let outMin = 0;
    let outMax = baseSensorDigital;

    switch (sensor) {
        case "Umidade_do_solo":
            umidadeSolo = parseFloat(mapValue(valorAD, outMin, outMax, inMin, inMax).toFixed(2));
            break;
        case "Temperatura_do_solo":
            temperaturaSolo = parseFloat(mapValue(valorAD, outMin, outMax, inMin, inMax).toFixed(2));
            break;
        case "pH_do_solo":
            pHSolo = parseFloat(mapValue(valorAD, outMin, outMax, inMin, inMax).toFixed(2));
            break;
        case "Nivel_de_nutrientes":
            nivelNutrientes = parseFloat(mapValue(valorAD, outMin, outMax, inMin, inMax).toFixed(2));
            break;
        case "Radiacao_solar":
            radiacaoSolar = parseFloat(mapValue(valorAD, outMin, outMax, inMin, inMax).toFixed(2));
            break;
        case "Umidade_relativa_do_ar":
            umidadeRelativa = parseFloat(mapValue(valorAD, outMin, outMax, inMin, inMax).toFixed(2));
            break;
        case "Temperatura_ambiente":
            temperaturaAmbiente = parseFloat(mapValue(valorAD, outMin, outMax, inMin, inMax).toFixed(2));
            break;
        case "Vazao":
            vazao = parseFloat(mapValue(valorAD, outMin, outMax, inMin, inMax).toFixed(2));
            break;
    }
}

// Reestrutura o payload com os campos mapeados
msg.payload = {
    "field1": umidadeSolo,
    "field2": temperaturaSolo,
    "field3": pHSolo,
    "field4": nivelNutrientes,
    "field5": radiacaoSolar,
    "field6": umidadeRelativa,
    "field7": temperaturaAmbiente,
    "field8": vazao
};

return msg;

**Código do Node Red em Python**

In [ ]:
 ## Código do NodeRed em Python:
 
def mapValue(value, in_min, in_max, out_min, out_max):
    return (value - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

for i in range(len(nr_data['data_config'])):
    data_config = nr_data['data_config'][i]
    data_sensores = nr_data['data_sensores'][i]
    sensor = data_config.get('sensor')

    if sensor == 'Umidade_do_solo':
        umidade_do_solo_in_min = data_config.get('Entrada_minima')
        umidade_do_solo_in_max = data_config.get('Entrada_maxima')
        umidade_do_solo_out_min = 0
        umidade_do_solo_out_max = nr_data['BASE_SENSOR_DIGITAL']
        umidade_do_solo_um = data_config.get('Entrada_um')
        umidade_do_solo_condicionado = data_config.get('Valor_condicionado')
        umidade_do_solo_valor= data_sensores.get('Conversor AD')
        umidade_do_solo_saida = mapValue(umidade_do_solo_valor, umidade_do_solo_out_min, umidade_do_solo_out_max, umidade_do_solo_in_min, umidade_do_solo_in_max)
        umidade_do_solo_saida = np.round(umidade_do_solo_saida,2)
        
    if sensor == 'Temperatura_do_solo':
        temperatura_do_solo_in_min = data_config.get('Entrada_minima')
        temperatura_do_solo_in_max = data_config.get('Entrada_maxima')
        temperatura_do_solo_out_min = 0
        temperatura_do_solo_out_max = nr_data['BASE_SENSOR_DIGITAL']
        temperatura_do_solo_um = data_config.get('Entrada_um')
        temperatura_do_solo_condicionado = data_config.get('Valor_condicionado')
        temperatura_do_solo_valor= data_sensores.get('Conversor AD')	
        temperatura_do_solo_saida = mapValue(temperatura_do_solo_valor, temperatura_do_solo_out_min, temperatura_do_solo_out_max, temperatura_do_solo_in_min,  temperatura_do_solo_in_max)
        temperatura_do_solo_saida = np.round(temperatura_do_solo_saida,2)
        
    if sensor == 'pH_do_solo':
        pH_do_solo_in_min = data_config.get('Entrada_minima')
        pH_do_solo_in_max = data_config.get('Entrada_maxima')
        pH_do_solo_out_min = 0
        pH_do_solo_out_max = nr_data['BASE_SENSOR_DIGITAL']
        pH_do_solo_um = data_config.get('Entrada_um')
        pH_do_solo_condicionado = data_config.get('Valor_condicionado')
        pH_do_solo_valor= data_sensores.get('Conversor AD')
        pH_do_solo_saida = mapValue(pH_do_solo_valor, pH_do_solo_out_min, pH_do_solo_out_max, pH_do_solo_in_min, pH_do_solo_in_max)
        pH_do_solo_saida = np.round(pH_do_solo_saida,2)
        
    if sensor == 'Nivel_de_nutrientes':
        nivel_de_nutrientes_in_min = data_config.get('Entrada_minima')
        nivel_de_nutrientes_in_max = data_config.get('Entrada_maxima')
        nivel_de_nutrientes_out_min = 0
        nivel_de_nutrientes_out_max = nr_data['BASE_SENSOR_DIGITAL']
        nivel_de_nutrientes_um = data_config.get('Entrada_um')
        nivel_de_nutrientes_condicionado = data_config.get('Valor_condicionado')
        nivel_de_nutrientes_valor= data_sensores.get('Conversor AD')
        nivel_de_nutrientes_saida = mapValue(nivel_de_nutrientes_valor, nivel_de_nutrientes_out_min, nivel_de_nutrientes_out_max, nivel_de_nutrientes_in_min, nivel_de_nutrientes_in_max)
        nivel_de_nutrientes_saida = np.round(nivel_de_nutrientes_saida,2)
        
        
    if sensor == 'Radiacao_solar':
        radiacao_solar_in_min = data_config.get('Entrada_minima')
        radiacao_solar_in_max = data_config.get('Entrada_maxima')
        radiacao_solar_out_min = 0
        radiacao_solar_out_max = nr_data['BASE_SENSOR_DIGITAL']
        radiacao_solar_um = data_config.get('Entrada_um')
        radiacao_solar_condicionado = data_config.get('Valor_condicionado')
        radiacao_solar_valor= data_sensores.get('Conversor AD')
        radiacao_solar_saida = mapValue(radiacao_solar_valor, radiacao_solar_out_min, radiacao_solar_out_max, radiacao_solar_in_min, radiacao_solar_in_max)
        radiacao_solar_saida= np.round(radiacao_solar_saida,2)
        
    if sensor == 'Umidade_relativa_do_ar':
        umidade_relativa_do_ar_in_min = data_config.get('Entrada_minima')
        umidade_relativa_do_ar_in_max = data_config.get('Entrada_maxima')
        umidade_relativa_do_ar_out_min = 0
        umidade_relativa_do_ar_out_max = nr_data['BASE_SENSOR_DIGITAL']
        umidade_relativa_do_ar_um = data_config.get('Entrada_um')
        umidade_relativa_do_ar_condicionado = data_config.get('Valor_condicionado')
        umidade_relativa_do_ar_valor= data_sensores.get('Conversor AD')
        umidade_relativa_do_ar_saida = mapValue(umidade_relativa_do_ar_valor, umidade_relativa_do_ar_out_min, umidade_relativa_do_ar_out_max, umidade_relativa_do_ar_in_min, umidade_relativa_do_ar_in_max)
        umidade_relativa_do_ar_saida = np.round(umidade_relativa_do_ar_saida,2)
        
    if sensor == 'Temperatura_ambiente':
        temperatura_ambiente_in_min = data_config.get('Entrada_minima')
        temperatura_ambiente_in_max = data_config.get('Entrada_maxima')
        temperatura_ambiente_out_min = 0
        temperatura_ambiente_out_max = nr_data['BASE_SENSOR_DIGITAL']
        temperatura_ambiente_um = data_config.get('Entrada_um')
        temperatura_ambiente_condicionado = data_config.get('Valor_condicionado')
        temperatura_ambiente_valor= data_sensores.get('Conversor AD')
        temperatura_ambiente_saida= mapValue(temperatura_ambiente_valor, temperatura_ambiente_out_min, temperatura_ambiente_out_max, temperatura_ambiente_in_min, temperatura_ambiente_in_max)
        temperatura_ambiente_saida= np.round(temperatura_ambiente_saida,2)
        
    if sensor == 'Vazao':
        vazao_in_min = data_config.get('Entrada_minima')
        vazao_in_max = data_config.get('Entrada_maxima')
        vazao_out_min = 0
        vazao_out_max = nr_data['BASE_SENSOR_DIGITAL']
        vazao_um = data_config.get('Entrada_um')
        vazao_condicionado = data_config.get('Valor_condicionado')
        vazao_valor= data_sensores.get('Conversor AD')
        vazao_saida = mapValue(vazao_valor, vazao_out_min, vazao_out_max, vazao_in_min, vazao_in_max)
        vazao_saida = np.round(vazao_saida,2)


msg_payload = {
    "field1": umidade_do_solo_saida,
    "field2": temperatura_do_solo_saida,
    "field3": pH_do_solo_saida,
    "field4": nivel_de_nutrientes_saida,
    "field5": radiacao_solar_saida,
    "field6": umidade_relativa_do_ar_saida,
    "field7": temperatura_ambiente_saida,
    "field8": vazao_saida
}


# Construção da Classe Sensor

In [288]:
import pandas as pd
import numpy as np
import requests
import json
import time
from dataclasses import dataclass
import pydantic
from tqdm import tqdm

In [289]:
@dataclass
class Sensores:
    sensores_config: dict = None
    sensores_values: dict = None
    sensores_config_init: dict = None
    df_sensores_config= None
    df_sensores_leitura= None
    BASE_DECIMAL: int = None  # Exemplo de valor padrão para inteiro
    json_data: dict= None
    df_sensores_leitura_min =None 
    df_sensores_leitura_max= None
    
    
    
    def sensores_config_example(self):
        config_example = {'Sensor': ["Temperatura_ambiente"], 
                          'Entrada_minima': [-20], 
                          'Entrada_maxima': [50], 
                          'Entrada_um': ['°C'], 
                          'Saida_minima': [0.2], 
                          'Saida_maxima': [1.4], 
                          'Saida_um': ['V'], 
                          'K': [3],
                          'K_um': ['V/ºC'],
                          'a': [1],
                          'a_um': ['V'],
                          'Modelo_matematico': ['"O(I)= 3V/ºC.I + 1V"'],
                          'Ganho': 4
                          }
        
        config_example_txt = ''' Sensor: Nome do sensor, 
                            Entrada_minima: Valor mínimo de leitura do sensor, 
                            Entrada_maxima: Valor máximo de leitura do sensor, 
                            Entrada_um: Unidade de medida de leitura do sensor, 
                            Saida_minima: Valor mínimo enviado pelo sensor, 
                            Saida_maxima: Valor máximo enviado pelo sensor, 
                            Saida_um: Unidade de medida enviada na saída do sensor, 
                            K: valor da constante do modelo matemático do sensor,
                            K_um: Unidade de medida de K ,
                            a': Valor do intercepto do modelo matemático do sensor,
                            a_um': Unidade de medida de a,
                            Modelo_matematico': modelo matemático do sensor,
                            Ganho: Ganho do conversro AD do sensor, 
                            
                          
                          '''
        df_config_example = pd.DataFrame(config_example)  # Corrigido para criar um DataFrame

        print(f'Dicionário de dados: \n\n {config_example_txt}')
        
        entrada_txt = {'Umidade_do_solo': {
                       'Entrada_minima': 0, 
                       'Entrada_maxima': 100, 
                       'Entrada_um': '%Vol', 
                       'Saida_minima': 0.3, 
                       'Saida_maxima': 1, 
                       'Saida_um': 'V', 
                       'Ganho': 5
                       }
                    }
        print(' ')
        print(f'Entrada: \n\n {entrada_txt}')
        print(' ')
        print('Saída: \n\n')
        return df_config_example
    
    def sensores_values_saida_example(self):
        data_saidas = {'Sensor':['Temperatura_ambiente'],
               'Valor_lido':[0],
               'Valor_lido_um':["ºC"],
               'Valor_enviado':[1],
               'Valor_enviado_um':['V'],
               'Valor_condicionado':[1.3],
               'Valor_condicionado_um':["V"],
               'Valor_decimal':[1024]    
            }
        
        data_saidas_txt = '''
                            Valor_lido: Valor lido pelo sensor,
                            Valor_lido_um: Unidade de medida da leitura do sensor,
                            Valor_enviado: Valor enviado pelo sensor,
                            Valor_enviado_um: Unidade de medida enviada pelo sensor,
                            Valor_condicionado: Valor enviado pelo sensor após passar pelo conversor AD,
                            Valor_condicionado_um: Unidade de medida do valor condicionado,
                            Valor_decimal: Valor enviado na base decimal     
                          
                          '''
        df_data_saidas= pd.DataFrame(data_saidas)
        entrada_txt= {
                        'Umidade_do_solo': 10,
                        'Temperatura_do_solo': 20,
                        'pH_do_solo': 7,
                        'Nivel_de_nutrientes': 300,
                        'Radiacao_solar': 1000,
                        'Umidade_relativa_do_ar': 55,
                        'Temperatura_ambiente': 25,
                        'Vazao': 1.5
                    }                
        
        print(f'Dicionário de dados: \n\n {data_saidas_txt}')
        print(' ')
        print(f'Entrada: \n\n {entrada_txt}')
        print(' ')
        print('Saída: \n\n')
        
        return df_data_saidas

    def sensores_config_initializer(self, sensores_config):
        data_sensores_config = {
        'sensor': list(sensores_config.keys()),
        'Entrada_minima': np.zeros(len(sensores_config.keys())).tolist(),
        'Entrada_maxima': np.zeros(len(sensores_config.keys())).tolist(),
        'Entrada_um': np.zeros(len(sensores_config.keys())).tolist(),
        'Saida_minima': np.zeros(len(sensores_config.keys())).tolist(),
        'Saida_maxima': np.zeros(len(sensores_config.keys())).tolist(),
        'Saida_um': np.zeros(len(sensores_config.keys())).tolist(),
        'K': np.zeros(len(sensores_config.keys())).tolist(),
        'K_um': np.zeros(len(sensores_config.keys())).tolist(),
        'a': np.zeros(len(sensores_config.keys())).tolist(),
        'a_um': np.zeros(len(sensores_config.keys())).tolist(),
        'Modelo_matematico': np.zeros(len(sensores_config.keys())).tolist(),
        'Ganho': np.zeros(len(sensores_config.keys())).tolist(),
        }
        self.df_sensores_config = pd.DataFrame(data_sensores_config)
        
        total_tarefas = len(list(sensores_config.keys()))

        # Cria a barra de progresso
        with tqdm(total=total_tarefas, desc="Sensores carregados", unit="sensores") as pbar:
            # Loop para preenchimento
            for sensor, config in sensores_config.items():
                print(sensor, config)
                self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'Entrada_minima'] = config['Entrada_minima']
                self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'Entrada_maxima'] = config['Entrada_maxima']
                self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'Entrada_um'] = config['Entrada_um']
                self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'Saida_minima'] = config['Saida_minima']
                self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'Saida_maxima'] = config['Saida_maxima']
                self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'Saida_um'] = config['Saida_um']
                
                # Cálculo dos valores de K e a
                K = (config['Saida_maxima'] - config['Saida_minima']) / (config['Entrada_maxima'] - config['Entrada_minima'])
                a = config['Saida_minima'] - (K * config['Entrada_minima'])
                
                self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'K']= K
                
                if '/' in config['Entrada_um']:
                    num = config['Entrada_um'].split('/')[0]
                    den = config['Entrada_um'].split('/')[1]
                    k_um=f"{config['Saida_um']}. {den} /{num}"
                    self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'K_um'] = k_um
                    
                else:  
                    k_um= f"{config['Saida_um']}/{config['Entrada_um']}"             
                    self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'K_um'] = k_um
                
                self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'a'] = a
                self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'a_um'] = config['Saida_um']
                
                # Preenchimento do modelo matemático
                self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'Modelo_matematico'] = f"O(I)={np.round(K, 3)} {k_um}.I + {np.round(a, 3)} {config['Saida_um']}"
                self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'Ganho'] = config['Ganho']
                
                
                time.sleep(0.2)  # Simula uma operação
                pbar.update(1)  # Atualiza a barra com cada tarefa concluída
            
        return self.df_sensores_config
    
    def add_sensor(self, sensor_config):
        # Adicionando novo sensor ao DataFrame
        sensor_name = list(sensor_config.keys())[0]
        #print(sensor_name)
        sensores = self.df_sensores_config['sensor'].values
        if sensor_name in sensores:
            return print(f"Sensor '{sensor_name}' já existe na lista.")
        
        else: 
            data_sensor_config = {
            'sensor': list(sensor_config.keys()),
            'Entrada_minima': np.zeros(len(sensor_config.keys())).tolist(),
            'Entrada_maxima': np.zeros(len(sensor_config.keys())).tolist(),
            'Entrada_um': np.zeros(len(sensor_config.keys())).tolist(),
            'Saida_minima': np.zeros(len(sensor_config.keys())).tolist(),
            'Saida_maxima': np.zeros(len(sensor_config.keys())).tolist(),
            'Saida_um': np.zeros(len(sensor_config.keys())).tolist(),
            'K': np.zeros(len(sensor_config.keys())).tolist(),
            'K_um': np.zeros(len(sensor_config.keys())).tolist(),
            'a': np.zeros(len(sensor_config.keys())).tolist(),
            'a_um': np.zeros(len(sensor_config.keys())).tolist(),
            'Modelo_matematico': np.zeros(len(sensor_config.keys())).tolist(),
            'Ganho': np.zeros(len(sensor_config.keys())).tolist(),
            }
            df_sensor_config = pd.DataFrame(data_sensor_config)
            
            total_tarefas = len(list(sensor_config.keys()))

            # Cria a barra de progresso
            with tqdm(total=total_tarefas, desc="Sensores carregados", unit="sensores") as pbar:
                # Loop para preenchimento
                for sensor, config in sensor_config.items():
                    print(sensor, config)
                    df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'Entrada_minima'] = config['Entrada_minima']
                    df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'Entrada_maxima'] = config['Entrada_maxima']
                    df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'Entrada_um'] = config['Entrada_um']
                    df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'Saida_minima'] = config['Saida_minima']
                    df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'Saida_maxima'] = config['Saida_maxima']
                    df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'Saida_um'] = config['Saida_um']
                    
                    # Cálculo dos valores de K e a
                    K = (config['Saida_maxima'] - config['Saida_minima']) / (config['Entrada_maxima'] - config['Entrada_minima'])
                    a = config['Saida_minima'] - (K * config['Entrada_minima'])
                    
                    df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'K'] = K
                    
                    
                    if '/' in config['Entrada_um']:
                        num = config['Entrada_um'].split('/')[0]
                        den = config['Entrada_um'].split('/')[1]
                        k_um = f"{config['Saida_um']}. {den} /{num}"
                        df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'K_um'] = f"{config['Saida_um']}. {den} /{num}"
                    
                    else:
                        k_um=  f"{config['Saida_um']}/{config['Entrada_um']}"               
                        df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'K_um'] = k_um
                    
                    df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'a'] = a
                    df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'a_um'] = config['Saida_um']
                    
                    # Preenchimento do modelo matemático
                    df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'Modelo_matematico'] = f"O(I)={np.round(K, 3)} {k_um}.I + {np.round(a, 3)} {config['Saida_um']}"
                    df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'Ganho'] = config['Ganho']
                    
                    time.sleep(0.2)  # Simula uma operação
                    pbar.update(1)  # Atualiza a barra com cada tarefa concluída
            

            
            # Adicionando a linha ao DataFrame
            self.df_sensores_config = pd.concat([self.df_sensores_config, df_sensor_config], ignore_index=True)

            return self.df_sensores_config
    
    def dell_sensor(self, sensor_name):
        self.df_sensores_config = self.df_sensores_config[self.df_sensores_config['sensor'] != sensor_name].reset_index(drop=True)
        return self.df_sensores_config
    
    
        
    def sensores_config_show(self):
        if self.df_sensores_config is None:
            return print('Por favor carrege o Json de inicialização dos sensores!')
        else:
            return self.df_sensores_config
        
    
    def _map_to_base_decimal(self, value, min_value, max_value, base_decimal=1023):
        """
        Mapeia um valor para o intervalo de 0 até base_decimal com base nos valores mínimo e máximo.

        Parameters:
        - value (float): O valor que será mapeado.
        - min_value (float): O valor mínimo esperado na escala original.
        - max_value (float): O valor máximo esperado na escala original.
        - base_decimal (int): O valor máximo na escala de destino (padrão é 1023).

        Returns:
        - int: O valor mapeado para o intervalo de 0 até base_decimal.
        """
        # Garantir que o valor esteja dentro do intervalo [min_value, max_value]
        if value < min_value:
            value = min_value
        elif value > max_value:
            value = max_value
        
        # Normalizar o valor para o intervalo de 0 a base_decimal
        normalized_value = (value - min_value) / (max_value - min_value)
        mapped_value = (normalized_value * base_decimal)
        
        return mapped_value
    
    def get_sensor_data(self, sensor_data, base_decimal=1023):
        self.BASE_DECIMAL = base_decimal
        data_saida = {'sensor':  list(sensor_data.keys()),
                        'Valor_lido':np.zeros(len(sensor_data.keys())).tolist(),
                        'Valor_lido_um':np.zeros(len(sensor_data.keys())).tolist(),
                        'Valor_enviado':np.zeros(len(sensor_data.keys())).tolist(),
                        'Valor_enviado_um':np.zeros(len(sensor_data.keys())).tolist(),
                        'Valor_condicionado':np.zeros(len(sensor_data.keys())).tolist(),
                        'Valor_condicionado_um':np.zeros(len(sensor_data.keys())).tolist(),
                        'Valor_decimal':np.zeros(len(sensor_data.keys())).tolist()    
                        }
        
        self.df_sensores_leitura = pd.DataFrame(data_saida)
    
        total_tarefas = len(list(sensor_data.keys()))

        # Cria a barra de progresso
        with tqdm(total=total_tarefas, desc="Sensores carregados", unit="sensores") as pbar:
            # Iterar sobre cada sensor e preencher os valores no DataFrame
            for sensor, valor_lido in sensor_data.items():
                # Definir o valor lido do sensor
                self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_lido'] = valor_lido

                
                # Definir o valor lido com a unidade de entrada
                self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_lido_um'] = \
                    self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'Entrada_um'].values[0]

                # Calcular o valor enviado
                self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_enviado'] = (self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'K'].values[0] * valor_lido) + self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'a'].values[0]

                # Definir a unidade do valor enviado
                self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_enviado_um'] = \
                    self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'Saida_um'].values[0]

                # Calcular o valor condicionado
                self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_condicionado'] = \
                    self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'Ganho'].values[0] * \
                    self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_enviado'].values[0]

                # Definir a unidade do valor condicionado
                self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_condicionado_um'] = \
                    self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'Saida_um'].values[0]

                # Calcular o valor decimal
                min_value= self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'Saida_minima'].values[0]* self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'Ganho'].values[0]
                max_value= self.df_sensores_config.loc[self .df_sensores_config['sensor'] == sensor, 'Saida_maxima'].values[0]* self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'Ganho'].values[0]
                #valor_enviado = self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_enviado'].values[0]
                valor_enviado = self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_condicionado'].values[0] #/ self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'Ganho'].values[0]
                self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_decimal'] = self._map_to_base_decimal(valor_enviado, min_value, max_value, base_decimal=self.BASE_DECIMAL)
                
                time.sleep(0.2)  # Simula uma operação
                pbar.update(1)  # Atualiza a barra com cada tarefa concluída
            
        # Exibir o DataFrame resultante
        return self.df_sensores_leitura

    
    def sensor_data_show(self):
        if self.df_sensores_leitura is None:
            return print('Por favor carrgue os dados dos sensores!')
            
        else:
            return self.df_sensores_leitura
        
    
    def sensor_min_max_test(self, min= True):
    
        if self.df_sensores_config is None:
            return print('Por favor carrege o Json de inicialização dos sensores!')
        
        else: 

            if self.df_sensores_leitura_min is None or self.df_sensores_leitura_max is None: 
                data_sensor_max = {}
                data_sensor_min= {}
                for s, vmin, vmax in zip(self.df_sensores_config['sensor'].values, self.df_sensores_config['Entrada_minima'].values, self.df_sensores_config['Entrada_maxima'].values ):  
                    data_sensor_min[s]= vmin
                    data_sensor_max[s]= vmax
                
                self.df_sensores_leitura_min = self.get_sensor_data(data_sensor_min, base_decimal=1023)
                self.df_sensores_leitura_max= self.get_sensor_data(data_sensor_max, base_decimal=1023)
                
        
        if min == True:
            return self.df_sensores_leitura_min
  
        if min==False: 
            return self.df_sensores_leitura_max
               
    
    def prepare_sensor_data(self):
        if self.df_sensores_config is None:
            return print('Por favor carrege o Json de inicialização dos sensores!')
        
        elif self.df_sensores_leitura is None:
            return print('Por favor carrgue os dados dos sensores!')
        
        elif self.BASE_DECIMAL is None:
            return print('Por favor carrgue o dado da Base decimal!')
        
        else:
            self.json_data= {}
            conf_dict = self.df_sensores_config.to_dict('records')
            data_senores= self.df_sensores_leitura.to_dict('records')
            self.json_data["data_sensores"]= data_senores
            self.json_data["data_config"]= conf_dict
            self.json_data["BASE_SENSOR_DIGITAL"]= self.BASE_DECIMAL
        
            return self.json_data
    #             1         2       3       4          5    
    def _mapValue(self, value, in_min, in_max, out_min , out_max):
        return np.round(((((value - in_min) * (out_max - out_min)) / (in_max - in_min)) + out_min),1)
    
    def show_sensor_response(self):
        total_tarefas = len(self.json_data['data_config'])
        saida = "Saída: \n\n"
        # Cria a barra de progresso
        with tqdm(total=total_tarefas, desc="Sensores carregados", unit="sensores") as pbar:
            # Loop para preenchimento
            for i in range(len(self.json_data['data_config'])):
                data_config = self.json_data['data_config'][i]
                data_sensores = self.json_data['data_sensores'][i]
                sensor = data_config.get('sensor')
                val_in_min = data_config.get('Entrada_minima')
                val_in_max = data_config.get('Entrada_maxima')
                val_out_min = 0
                val_out_max = self.BASE_DECIMAL
                um = data_config.get('Entrada_um')
                valor_decimal= data_sensores.get('Valor_decimal')
                #                            1             2            3             4           5
                valor_saida = self._mapValue(valor_decimal, val_out_min, val_out_max, val_in_min, val_in_max)
                #valor_saida = np.round(valor_saida,2)
                time.sleep(0.2)  # Simula uma operação
                pbar.update(1)  # Atualiza a barra com cada tarefa concluída
                
                saida = saida + f'{sensor}: {valor_saida} {um} \n'
        print(' ')
        print(saida)
        
    def show_sensor_response_from_sensor_data(self, sensor_data, base_decimal=1023):
        if self.df_sensores_config is None:
            return print('Por favor carrege o Json de inicialização dos sensores!')
            
        else:
            self.df_sensores_leitura= self.get_sensor_data(sensor_data, base_decimal=base_decimal)
            self.json_data = self.prepare_sensor_data()
            total_tarefas = len(self.json_data['data_config'])
            saida = "Saída: \n\n"
            # Cria a barra de progresso
            with tqdm(total=total_tarefas, desc="Sensores carregados", unit="sensores") as pbar:
                # Loop para preenchimento
                for i in range(len(self.json_data['data_config'])):
                    data_config = self.json_data['data_config'][i]
                    data_sensores = self.json_data['data_sensores'][i]
                    sensor = data_config.get('sensor')
                    val_in_min = data_config.get('Entrada_minima')
                    val_in_max = data_config.get('Entrada_maxima')
                    val_out_min = 0
                    val_out_max = json_data['BASE_SENSOR_DIGITAL']
                    um = data_config.get('Entrada_um')
                    valor_decimal= data_sensores.get('Valor_decimal')
                    valor_saida = self._mapValue(valor_decimal, val_out_min, val_out_max, val_in_min, val_in_max)
                    #valor_saida = np.round(valor_saida,2)
                    time.sleep(0.2)  # Simula uma operação
                    pbar.update(1)  # Atualiza a barra com cada tarefa concluída
                    
                    saida = saida + f'{sensor}: {valor_saida} {um} \n'
            print(' ')
            print(saida)
                
    def send_data_to_NodeRed(self, url, verbose=False):
        if self.df_sensores_config is None:
            return print('Por favor carrege o Json de inicialização dos sensores!')
        
        elif self.df_sensores_leitura is None:
            return print('Por favor carrgue os dados dos sensores!')
        
        elif self.BASE_DECIMAL is None:
            return print('Por favor carrgue o dado da Base decimal!')
        
        elif self.json_data is None:
            return print('Por favor carregue o areuivo de envio dos dados!')
        
        else:
            # Enviar os dados
            response = requests.post(url, json= self.json_data)

            # Verificar a resposta
            if response.status_code == 200:
               print("Dados enviados com sucesso!")
               
               if verbose==True:
                   self.show_sensor_response()
                
               
            
            else:
                print(f"Erro ao enviar dados: {response.status_code}")       
        

    def send_data_to_NodeRed_from_sensor_data(self, sensor_data, base_decimal, url, verbose=False):
        if self.df_sensores_config is None:
            return print('Por favor carrege o Json de inicialização dos sensores!')
        
        else:
            self.df_sensores_leitura= self.get_sensor_data(sensor_data, base_decimal=base_decimal)
            self.json_data = self.prepare_sensor_data()
            self.json_data= self.prepare_sensor_data()
           
           
            # Enviar os dados
            response = requests.post(url, json=json_data)

            # Verificar a resposta
            if response.status_code == 200:
                print("Dados enviados com sucesso!")
                
                if verbose==True:
                   self.show_sensor_response()
            
            else:
                print(f"Erro ao enviar dados: {response.status_code}")       
        
        


In [290]:
sensores = Sensores()

In [291]:
sensores.sensores_config_example()

Dicionário de dados: 

  Sensor: Nome do sensor, 
                            Entrada_minima: Valor mínimo de leitura do sensor, 
                            Entrada_maxima: Valor máximo de leitura do sensor, 
                            Entrada_um: Unidade de medida de leitura do sensor, 
                            Saida_minima: Valor mínimo enviado pelo sensor, 
                            Saida_maxima: Valor máximo enviado pelo sensor, 
                            Saida_um: Unidade de medida enviada na saída do sensor, 
                            K: valor da constante do modelo matemático do sensor,
                            K_um: Unidade de medida de K ,
                            a': Valor do intercepto do modelo matemático do sensor,
                            a_um': Unidade de medida de a,
                            Modelo_matematico': modelo matemático do sensor,
                            Ganho: Ganho do conversro AD do sensor, 
                            
           

,Sensor,Entrada_minima,Entrada_maxima,Entrada_um,Saida_minima,Saida_maxima,Saida_um,K,K_um,a,a_um,Modelo_matematico,Ganho
0,Temperatura_ambiente,-20,50,°C,0.2,1.4,V,3,V/ºC,1,V,"""O(I)= 3V/ºC.I + 1V""",4


In [292]:
sensores.sensores_config_example()

Dicionário de dados: 

  Sensor: Nome do sensor, 
                            Entrada_minima: Valor mínimo de leitura do sensor, 
                            Entrada_maxima: Valor máximo de leitura do sensor, 
                            Entrada_um: Unidade de medida de leitura do sensor, 
                            Saida_minima: Valor mínimo enviado pelo sensor, 
                            Saida_maxima: Valor máximo enviado pelo sensor, 
                            Saida_um: Unidade de medida enviada na saída do sensor, 
                            K: valor da constante do modelo matemático do sensor,
                            K_um: Unidade de medida de K ,
                            a': Valor do intercepto do modelo matemático do sensor,
                            a_um': Unidade de medida de a,
                            Modelo_matematico': modelo matemático do sensor,
                            Ganho: Ganho do conversro AD do sensor, 
                            
           

,Sensor,Entrada_minima,Entrada_maxima,Entrada_um,Saida_minima,Saida_maxima,Saida_um,K,K_um,a,a_um,Modelo_matematico,Ganho
0,Temperatura_ambiente,-20,50,°C,0.2,1.4,V,3,V/ºC,1,V,"""O(I)= 3V/ºC.I + 1V""",4


In [293]:
sensores.sensores_config_example()

Dicionário de dados: 

  Sensor: Nome do sensor, 
                            Entrada_minima: Valor mínimo de leitura do sensor, 
                            Entrada_maxima: Valor máximo de leitura do sensor, 
                            Entrada_um: Unidade de medida de leitura do sensor, 
                            Saida_minima: Valor mínimo enviado pelo sensor, 
                            Saida_maxima: Valor máximo enviado pelo sensor, 
                            Saida_um: Unidade de medida enviada na saída do sensor, 
                            K: valor da constante do modelo matemático do sensor,
                            K_um: Unidade de medida de K ,
                            a': Valor do intercepto do modelo matemático do sensor,
                            a_um': Unidade de medida de a,
                            Modelo_matematico': modelo matemático do sensor,
                            Ganho: Ganho do conversro AD do sensor, 
                            
           

,Sensor,Entrada_minima,Entrada_maxima,Entrada_um,Saida_minima,Saida_maxima,Saida_um,K,K_um,a,a_um,Modelo_matematico,Ganho
0,Temperatura_ambiente,-20,50,°C,0.2,1.4,V,3,V/ºC,1,V,"""O(I)= 3V/ºC.I + 1V""",4


In [294]:
sensores.sensores_values_saida_example()

Dicionário de dados: 

 
                            Valor_lido: Valor lido pelo sensor,
                            Valor_lido_um: Unidade de medida da leitura do sensor,
                            Valor_enviado: Valor enviado pelo sensor,
                            Valor_enviado_um: Unidade de medida enviada pelo sensor,
                            Valor_condicionado: Valor enviado pelo sensor após passar pelo conversor AD,
                            Valor_condicionado_um: Unidade de medida do valor condicionado,
                            Valor_decimal: Valor enviado na base decimal     
                          
                          
 
Entrada: 

 {'Umidade_do_solo': 10, 'Temperatura_do_solo': 20, 'pH_do_solo': 7, 'Nivel_de_nutrientes': 300, 'Radiacao_solar': 1000, 'Umidade_relativa_do_ar': 55, 'Temperatura_ambiente': 25, 'Vazao': 1.5}
 
Saída: 




,Sensor,Valor_lido,Valor_lido_um,Valor_enviado,Valor_enviado_um,Valor_condicionado,Valor_condicionado_um,Valor_decimal
0,Temperatura_ambiente,0,ºC,1,V,1.3,V,1024


In [295]:
# Configurações específicas para cada sensor
sensores_config = {
    'Umidade_do_solo': {'Entrada_minima': 0, 'Entrada_maxima': 100, 'Entrada_um': '%Vol', 'Saida_minima': 0.3, 'Saida_maxima': 1, 'Saida_um': 'V', 'Ganho': 5},
    'Temperatura_do_solo': {'Entrada_minima': -35, 'Entrada_maxima': 50, 'Entrada_um': '°C', 'Saida_minima': 0, 'Saida_maxima': 5, 'Saida_um': 'V', 'Ganho': 2.5},
    'pH_do_solo': {'Entrada_minima': 0, 'Entrada_maxima': 14, 'Entrada_um': 'pH', 'Saida_minima': 0, 'Saida_maxima': 0.4, 'Saida_um': 'V', 'Ganho': 1.667},
    'Nivel_de_nutrientes': {'Entrada_minima': 0, 'Entrada_maxima': 100, 'Entrada_um': 'ppm', 'Saida_minima': 0, 'Saida_maxima': 5, 'Saida_um': 'V', 'Ganho': 1.250},
    'Radiacao_solar': {'Entrada_minima': 0, 'Entrada_maxima': 3000, 'Entrada_um': 'W/m²', 'Saida_minima': 0, 'Saida_maxima': 10, 'Saida_um': 'V', 'Ganho': 1},
    'Umidade_relativa_do_ar': {'Entrada_minima': 0, 'Entrada_maxima': 100, 'Entrada_um': '%', 'Saida_minima': 0, 'Saida_maxima': 3.3, 'Saida_um': 'V', 'Ganho': 0.833},
    'Temperatura_ambiente': {'Entrada_minima': -55, 'Entrada_maxima': 150, 'Entrada_um': '°C', 'Saida_minima': 0, 'Saida_maxima': 2.1, 'Saida_um': 'V', 'Ganho': 0.714}
    }

In [296]:
sensores.sensores_config_show()

Por favor carrege o Json de inicialização dos sensores!


In [297]:
df_config = sensores.sensores_config_initializer(sensores_config)
df_config

Sensores carregados:   0%|          | 0/7 [00:00<?, ?sensores/s]C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:135: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '%Vol' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'Entrada_um'] = config['Entrada_um']
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:138: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df_sensores_config.loc[self.df_sensores_config['sensor'] == sensor, 'Saida_um'] = config['Saida_um']
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:154: FutureWarning: Setting an item of incompatible dtype is deprecated 

Umidade_do_solo {'Entrada_minima': 0, 'Entrada_maxima': 100, 'Entrada_um': '%Vol', 'Saida_minima': 0.3, 'Saida_maxima': 1, 'Saida_um': 'V', 'Ganho': 5}


Sensores carregados:  14%|█▍        | 1/7 [00:00<00:01,  3.99sensores/s]

Temperatura_do_solo {'Entrada_minima': -35, 'Entrada_maxima': 50, 'Entrada_um': '°C', 'Saida_minima': 0, 'Saida_maxima': 5, 'Saida_um': 'V', 'Ganho': 2.5}


Sensores carregados:  29%|██▊       | 2/7 [00:00<00:01,  3.75sensores/s]

...

Umidade_relativa_do_ar {'Entrada_minima': 0, 'Entrada_maxima': 100, 'Entrada_um': '%', 'Saida_minima': 0, 'Saida_maxima': 3.3, 'Saida_um': 'V', 'Ganho': 0.833}


Sensores carregados:  86%|████████▌ | 6/7 [00:01<00:00,  3.94sensores/s]

Temperatura_ambiente {'Entrada_minima': -55, 'Entrada_maxima': 150, 'Entrada_um': '°C', 'Saida_minima': 0, 'Saida_maxima': 2.1, 'Saida_um': 'V', 'Ganho': 0.714}


Sensores carregados: 100%|██████████| 7/7 [00:01<00:00,  3.83sensores/s]


,sensor,Entrada_minima,Entrada_maxima,Entrada_um,Saida_minima,Saida_maxima,Saida_um,K,K_um,a,a_um,Modelo_matematico,Ganho
0,Umidade_do_solo,0.0,100.0,%Vol,0.3,1.0,V,0.007000,V/%Vol,0.300000,V,O(I)=0.007 V/%Vol.I + 0.3 V,5.000
1,Temperatura_do_solo,-35.0,50.0,°C,0.0,5.0,V,0.058824,V/°C,2.058824,V,O(I)=0.059 V/°C.I + 2.059 V,2.500
2,pH_do_solo,0.0,14.0,pH,0.0,0.4,V,0.028571,V/pH,0.000000,V,O(I)=0.029 V/pH.I + 0.0 V,1.667
3,Nivel_de_nutrientes,0.0,100.0,ppm,0.0,5.0,V,0.050000,V/ppm,0.000000,V,O(I)=0.05 V/ppm.I + 0.0 V,1.250
4,Radiacao_solar,0.0,3000.0,W/m²,0.0,10.0,V,0.003333,V. m² /W,0.000000,V,O(I)=0.003 V. m² /W.I + 0.0 V,1.000
5,Umidade_relativa_do_ar,0.0,100.0,%,0.0,3.3,V,0.033000,V/%,0.000000,V,O(I)=0.033 V/%.I + 0.0 V,0.833
6,Temperatura_ambiente,-55.0,150.0,°C,0.0,2.1,V,0.010244,V/°C,0.563415,V,O(I)=0.01 V/°C.I + 0.563 V,0.714


In [298]:
sensores.sensores_config_show()

,sensor,Entrada_minima,Entrada_maxima,Entrada_um,Saida_minima,Saida_maxima,Saida_um,K,K_um,a,a_um,Modelo_matematico,Ganho
0,Umidade_do_solo,0.0,100.0,%Vol,0.3,1.0,V,0.007000,V/%Vol,0.300000,V,O(I)=0.007 V/%Vol.I + 0.3 V,5.000
1,Temperatura_do_solo,-35.0,50.0,°C,0.0,5.0,V,0.058824,V/°C,2.058824,V,O(I)=0.059 V/°C.I + 2.059 V,2.500
2,pH_do_solo,0.0,14.0,pH,0.0,0.4,V,0.028571,V/pH,0.000000,V,O(I)=0.029 V/pH.I + 0.0 V,1.667
3,Nivel_de_nutrientes,0.0,100.0,ppm,0.0,5.0,V,0.050000,V/ppm,0.000000,V,O(I)=0.05 V/ppm.I + 0.0 V,1.250
4,Radiacao_solar,0.0,3000.0,W/m²,0.0,10.0,V,0.003333,V. m² /W,0.000000,V,O(I)=0.003 V. m² /W.I + 0.0 V,1.000
5,Umidade_relativa_do_ar,0.0,100.0,%,0.0,3.3,V,0.033000,V/%,0.000000,V,O(I)=0.033 V/%.I + 0.0 V,0.833
6,Temperatura_ambiente,-55.0,150.0,°C,0.0,2.1,V,0.010244,V/°C,0.563415,V,O(I)=0.01 V/°C.I + 0.563 V,0.714


In [299]:
sensores.df_sensores_config

,sensor,Entrada_minima,Entrada_maxima,Entrada_um,Saida_minima,Saida_maxima,Saida_um,K,K_um,a,a_um,Modelo_matematico,Ganho
0,Umidade_do_solo,0.0,100.0,%Vol,0.3,1.0,V,0.007000,V/%Vol,0.300000,V,O(I)=0.007 V/%Vol.I + 0.3 V,5.000
1,Temperatura_do_solo,-35.0,50.0,°C,0.0,5.0,V,0.058824,V/°C,2.058824,V,O(I)=0.059 V/°C.I + 2.059 V,2.500
2,pH_do_solo,0.0,14.0,pH,0.0,0.4,V,0.028571,V/pH,0.000000,V,O(I)=0.029 V/pH.I + 0.0 V,1.667
3,Nivel_de_nutrientes,0.0,100.0,ppm,0.0,5.0,V,0.050000,V/ppm,0.000000,V,O(I)=0.05 V/ppm.I + 0.0 V,1.250
4,Radiacao_solar,0.0,3000.0,W/m²,0.0,10.0,V,0.003333,V. m² /W,0.000000,V,O(I)=0.003 V. m² /W.I + 0.0 V,1.000
5,Umidade_relativa_do_ar,0.0,100.0,%,0.0,3.3,V,0.033000,V/%,0.000000,V,O(I)=0.033 V/%.I + 0.0 V,0.833
6,Temperatura_ambiente,-55.0,150.0,°C,0.0,2.1,V,0.010244,V/°C,0.563415,V,O(I)=0.01 V/°C.I + 0.563 V,0.714


In [300]:
sensor_config= {'Vazao': {'Entrada_minima': 0, 'Entrada_maxima': 1200, 'Entrada_um': 'L/min', 'Saida_minima': 1, 'Saida_maxima': 5, 'Saida_um': 'V', 'Ganho': 0.625}}

In [301]:
df_sensor_conf= sensores.add_sensor(sensor_config)
df_sensor_conf

Sensores carregados:   0%|          | 0/1 [00:00<?, ?sensores/s]

C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:204: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'L/min' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'Entrada_um'] = config['Entrada_um']
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:207: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'Saida_um'] = config['Saida_um']
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:220: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V. min /L' has dtype incompat

Vazao {'Entrada_minima': 0, 'Entrada_maxima': 1200, 'Entrada_um': 'L/min', 'Saida_minima': 1, 'Saida_maxima': 5, 'Saida_um': 'V', 'Ganho': 0.625}


Sensores carregados: 100%|██████████| 1/1 [00:00<00:00,  4.32sensores/s]


,sensor,Entrada_minima,Entrada_maxima,Entrada_um,Saida_minima,Saida_maxima,Saida_um,K,K_um,a,a_um,Modelo_matematico,Ganho
0,Umidade_do_solo,0.0,100.0,%Vol,0.3,1.0,V,0.007000,V/%Vol,0.300000,V,O(I)=0.007 V/%Vol.I + 0.3 V,5.000
1,Temperatura_do_solo,-35.0,50.0,°C,0.0,5.0,V,0.058824,V/°C,2.058824,V,O(I)=0.059 V/°C.I + 2.059 V,2.500
2,pH_do_solo,0.0,14.0,pH,0.0,0.4,V,0.028571,V/pH,0.000000,V,O(I)=0.029 V/pH.I + 0.0 V,1.667
3,Nivel_de_nutrientes,0.0,100.0,ppm,0.0,5.0,V,0.050000,V/ppm,0.000000,V,O(I)=0.05 V/ppm.I + 0.0 V,1.250
4,Radiacao_solar,0.0,3000.0,W/m²,0.0,10.0,V,0.003333,V. m² /W,0.000000,V,O(I)=0.003 V. m² /W.I + 0.0 V,1.000
5,Umidade_relativa_do_ar,0.0,100.0,%,0.0,3.3,V,0.033000,V/%,0.000000,V,O(I)=0.033 V/%.I + 0.0 V,0.833
6,Temperatura_ambiente,-55.0,150.0,°C,0.0,2.1,V,0.010244,V/°C,0.563415,V,O(I)=0.01 V/°C.I + 0.563 V,0.714
7,Vazao,0.0,1200.0,L/min,1.0,5.0,V,0.003333,V. min /L,1.000000,V,O(I)=0.003 V. min /L.I + 1.0 V,0.625


In [302]:
df_sensor_conf= sensores.add_sensor(sensor_config)
df_sensor_conf

Sensor 'Vazao' já existe na lista.


In [308]:
sensores.sensores_config_show()

,sensor,Entrada_minima,Entrada_maxima,Entrada_um,Saida_minima,Saida_maxima,Saida_um,K,K_um,a,a_um,Modelo_matematico,Ganho
0,Umidade_do_solo,0.0,100.0,%Vol,0.3,1.0,V,0.007000,V/%Vol,0.300000,V,O(I)=0.007 V/%Vol.I + 0.3 V,5.000
1,Temperatura_do_solo,-35.0,50.0,°C,0.0,5.0,V,0.058824,V/°C,2.058824,V,O(I)=0.059 V/°C.I + 2.059 V,2.500
2,pH_do_solo,0.0,14.0,pH,0.0,0.4,V,0.028571,V/pH,0.000000,V,O(I)=0.029 V/pH.I + 0.0 V,1.667
3,Nivel_de_nutrientes,0.0,100.0,ppm,0.0,5.0,V,0.050000,V/ppm,0.000000,V,O(I)=0.05 V/ppm.I + 0.0 V,1.250
4,Radiacao_solar,0.0,3000.0,W/m²,0.0,10.0,V,0.003333,V. m² /W,0.000000,V,O(I)=0.003 V. m² /W.I + 0.0 V,1.000
5,Umidade_relativa_do_ar,0.0,100.0,%,0.0,3.3,V,0.033000,V/%,0.000000,V,O(I)=0.033 V/%.I + 0.0 V,0.833
6,Temperatura_ambiente,-55.0,150.0,°C,0.0,2.1,V,0.010244,V/°C,0.563415,V,O(I)=0.01 V/°C.I + 0.563 V,0.714
7,Vazao,0.0,1200.0,L/min,1.0,5.0,V,0.003333,V. min /L,1.000000,V,O(I)=0.003 V. min /L.I + 1.0 V,0.625


In [304]:
sensor_conf3 = sensores.dell_sensor("Vazao")
sensor_conf3

,sensor,Entrada_minima,Entrada_maxima,Entrada_um,Saida_minima,Saida_maxima,Saida_um,K,K_um,a,a_um,Modelo_matematico,Ganho
0,Umidade_do_solo,0.0,100.0,%Vol,0.3,1.0,V,0.007000,V/%Vol,0.300000,V,O(I)=0.007 V/%Vol.I + 0.3 V,5.000
1,Temperatura_do_solo,-35.0,50.0,°C,0.0,5.0,V,0.058824,V/°C,2.058824,V,O(I)=0.059 V/°C.I + 2.059 V,2.500
2,pH_do_solo,0.0,14.0,pH,0.0,0.4,V,0.028571,V/pH,0.000000,V,O(I)=0.029 V/pH.I + 0.0 V,1.667
3,Nivel_de_nutrientes,0.0,100.0,ppm,0.0,5.0,V,0.050000,V/ppm,0.000000,V,O(I)=0.05 V/ppm.I + 0.0 V,1.250
4,Radiacao_solar,0.0,3000.0,W/m²,0.0,10.0,V,0.003333,V. m² /W,0.000000,V,O(I)=0.003 V. m² /W.I + 0.0 V,1.000
5,Umidade_relativa_do_ar,0.0,100.0,%,0.0,3.3,V,0.033000,V/%,0.000000,V,O(I)=0.033 V/%.I + 0.0 V,0.833
6,Temperatura_ambiente,-55.0,150.0,°C,0.0,2.1,V,0.010244,V/°C,0.563415,V,O(I)=0.01 V/°C.I + 0.563 V,0.714


In [305]:
df_sensor_conf= sensores.add_sensor(sensor_config)
df_sensor_conf

Sensores carregados:   0%|          | 0/1 [00:00<?, ?sensores/s]C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:204: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'L/min' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'Entrada_um'] = config['Entrada_um']
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:207: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sensor_config.loc[df_sensor_config['sensor'] == sensor, 'Saida_um'] = config['Saida_um']
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:220: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in 

Vazao {'Entrada_minima': 0, 'Entrada_maxima': 1200, 'Entrada_um': 'L/min', 'Saida_minima': 1, 'Saida_maxima': 5, 'Saida_um': 'V', 'Ganho': 0.625}


Sensores carregados: 100%|██████████| 1/1 [00:00<00:00,  4.10sensores/s]


,sensor,Entrada_minima,Entrada_maxima,Entrada_um,Saida_minima,Saida_maxima,Saida_um,K,K_um,a,a_um,Modelo_matematico,Ganho
0,Umidade_do_solo,0.0,100.0,%Vol,0.3,1.0,V,0.007000,V/%Vol,0.300000,V,O(I)=0.007 V/%Vol.I + 0.3 V,5.000
1,Temperatura_do_solo,-35.0,50.0,°C,0.0,5.0,V,0.058824,V/°C,2.058824,V,O(I)=0.059 V/°C.I + 2.059 V,2.500
2,pH_do_solo,0.0,14.0,pH,0.0,0.4,V,0.028571,V/pH,0.000000,V,O(I)=0.029 V/pH.I + 0.0 V,1.667
3,Nivel_de_nutrientes,0.0,100.0,ppm,0.0,5.0,V,0.050000,V/ppm,0.000000,V,O(I)=0.05 V/ppm.I + 0.0 V,1.250
4,Radiacao_solar,0.0,3000.0,W/m²,0.0,10.0,V,0.003333,V. m² /W,0.000000,V,O(I)=0.003 V. m² /W.I + 0.0 V,1.000
5,Umidade_relativa_do_ar,0.0,100.0,%,0.0,3.3,V,0.033000,V/%,0.000000,V,O(I)=0.033 V/%.I + 0.0 V,0.833
6,Temperatura_ambiente,-55.0,150.0,°C,0.0,2.1,V,0.010244,V/°C,0.563415,V,O(I)=0.01 V/°C.I + 0.563 V,0.714
7,Vazao,0.0,1200.0,L/min,1.0,5.0,V,0.003333,V. min /L,1.000000,V,O(I)=0.003 V. min /L.I + 1.0 V,0.625


In [307]:
sensores.sensor_data_show()

Por favor carrgue os dados dos sensores!


In [309]:
# Lista de valores lidos para cada sensor (substitua pelos valores corretos)
sensor_data= {
    'Umidade_do_solo': 0,
    'Temperatura_do_solo': -35,
    'pH_do_solo': 0,
    'Nivel_de_nutrientes': 0,
    'Radiacao_solar': 0,
    'Umidade_relativa_do_ar': 0,
    'Temperatura_ambiente': -55,
    'Vazao': 0
}

In [310]:
df_leitura= sensores.get_sensor_data(sensor_data, base_decimal=1023)
df_leitura

Sensores carregados:   0%|          | 0/8 [00:00<?, ?sensores/s]C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:306: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '%Vol' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_lido_um'] = \
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:313: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_enviado_um'] = \
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:322: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an err

,sensor,Valor_lido,Valor_lido_um,Valor_enviado,Valor_enviado_um,Valor_condicionado,Valor_condicionado_um,Valor_decimal
0,Umidade_do_solo,0.0,%Vol,0.3,V,1.500,V,0.0
1,Temperatura_do_solo,-35.0,°C,0.0,V,0.000,V,0.0
2,pH_do_solo,0.0,pH,0.0,V,0.000,V,0.0
3,Nivel_de_nutrientes,0.0,ppm,0.0,V,0.000,V,0.0
4,Radiacao_solar,0.0,W/m²,0.0,V,0.000,V,0.0
5,Umidade_relativa_do_ar,0.0,%,0.0,V,0.000,V,0.0
6,Temperatura_ambiente,-55.0,°C,0.0,V,0.000,V,0.0
7,Vazao,0.0,L/min,1.0,V,0.625,V,0.0


In [311]:
sensores.sensor_data_show()

,sensor,Valor_lido,Valor_lido_um,Valor_enviado,Valor_enviado_um,Valor_condicionado,Valor_condicionado_um,Valor_decimal
0,Umidade_do_solo,0.0,%Vol,0.3,V,1.500,V,0.0
1,Temperatura_do_solo,-35.0,°C,0.0,V,0.000,V,0.0
2,pH_do_solo,0.0,pH,0.0,V,0.000,V,0.0
3,Nivel_de_nutrientes,0.0,ppm,0.0,V,0.000,V,0.0
4,Radiacao_solar,0.0,W/m²,0.0,V,0.000,V,0.0
5,Umidade_relativa_do_ar,0.0,%,0.0,V,0.000,V,0.0
6,Temperatura_ambiente,-55.0,°C,0.0,V,0.000,V,0.0
7,Vazao,0.0,L/min,1.0,V,0.625,V,0.0


In [312]:
sensores_min= sensores.sensor_min_max_test(min=True)
sensores_min

Sensores carregados:   0%|          | 0/8 [00:00<?, ?sensores/s]

C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:306: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '%Vol' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_lido_um'] = \
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:313: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_enviado_um'] = \
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:322: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatib

,sensor,Valor_lido,Valor_lido_um,Valor_enviado,Valor_enviado_um,Valor_condicionado,Valor_condicionado_um,Valor_decimal
0,Umidade_do_solo,0.0,%Vol,0.3,V,1.500,V,0.0
1,Temperatura_do_solo,-35.0,°C,0.0,V,0.000,V,0.0
2,pH_do_solo,0.0,pH,0.0,V,0.000,V,0.0
3,Nivel_de_nutrientes,0.0,ppm,0.0,V,0.000,V,0.0
4,Radiacao_solar,0.0,W/m²,0.0,V,0.000,V,0.0
5,Umidade_relativa_do_ar,0.0,%,0.0,V,0.000,V,0.0
6,Temperatura_ambiente,-55.0,°C,0.0,V,0.000,V,0.0
7,Vazao,0.0,L/min,1.0,V,0.625,V,0.0


In [313]:
sensores_max= sensores.sensor_min_max_test(min=False)
sensores_max

,sensor,Valor_lido,Valor_lido_um,Valor_enviado,Valor_enviado_um,Valor_condicionado,Valor_condicionado_um,Valor_decimal
0,Umidade_do_solo,100.0,%Vol,1.0,V,5.0000,V,1023.0
1,Temperatura_do_solo,50.0,°C,5.0,V,12.5000,V,1023.0
2,pH_do_solo,14.0,pH,0.4,V,0.6668,V,1023.0
3,Nivel_de_nutrientes,100.0,ppm,5.0,V,6.2500,V,1023.0
4,Radiacao_solar,3000.0,W/m²,10.0,V,10.0000,V,1023.0
5,Umidade_relativa_do_ar,100.0,%,3.3,V,2.7489,V,1023.0
6,Temperatura_ambiente,150.0,°C,2.1,V,1.4994,V,1023.0
7,Vazao,1200.0,L/min,5.0,V,3.1250,V,1023.0


In [314]:
json_data= sensores.prepare_sensor_data()
json_data

{'data_sensores': [{'sensor': 'Umidade_do_solo',
   'Valor_lido': 100.0,
   'Valor_lido_um': '%Vol',
   'Valor_enviado': 1.0,
   'Valor_enviado_um': 'V',
   'Valor_condicionado': 5.0,
   'Valor_condicionado_um': 'V',
   'Valor_decimal': 1023.0},
  {'sensor': 'Temperatura_do_solo',
   'Valor_lido': 50.0,
   'Valor_lido_um': '°C',
   'Valor_enviado': 5.0,
   'Valor_enviado_um': 'V',
   'Valor_condicionado': 12.5,
   'Valor_condicionado_um': 'V',
   'Valor_decimal': 1023.0},
  {'sensor': 'pH_do_solo',
   'Valor_lido': 14.0,
   'Valor_lido_um': 'pH',
   'Valor_enviado': 0.4,
   'Valor_enviado_um': 'V',
   'Valor_condicionado': 0.6668000000000001,
   'Valor_condicionado_um': 'V',
   'Valor_decimal': 1023.0},
  {'sensor': 'Nivel_de_nutrientes',
   'Valor_lido': 100.0,
   'Valor_lido_um': 'ppm',
   'Valor_enviado': 5.0,
   'Valor_enviado_um': 'V',
   'Valor_condicionado': 6.25,
   'Valor_condicionado_um': 'V',
   'Valor_decimal': 1023.0},
  {'sensor': 'Radiacao_solar',
   'Valor_lido': 3000.0

In [315]:
sensores.show_sensor_response()

Sensores carregados: 100%|██████████| 8/8 [00:01<00:00,  4.89sensores/s]

 
Saída: 

Umidade_do_solo: 100.0 %Vol 
Temperatura_do_solo: 50.0 °C 
pH_do_solo: 14.0 pH 
Nivel_de_nutrientes: 100.0 ppm 
Radiacao_solar: 3000.0 W/m² 
Umidade_relativa_do_ar: 100.0 % 
Temperatura_ambiente: 150.0 °C 
Vazao: 1200.0 L/min 



In [316]:
# Lista de valores lidos para cada sensor (substitua pelos valores corretos)
sensor_data= {
    'Umidade_do_solo': 0,
    'Temperatura_do_solo': -35,
    'pH_do_solo': 0,
    'Nivel_de_nutrientes': 0,
    'Radiacao_solar': 0,
    'Umidade_relativa_do_ar': 0,
    'Temperatura_ambiente': -55,
    'Vazao': 0
}

In [317]:
sensores.show_sensor_response_from_sensor_data(sensor_data, base_decimal=1023)

Sensores carregados:   0%|          | 0/8 [00:00<?, ?sensores/s]C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:306: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '%Vol' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_lido_um'] = \
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:313: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_enviado_um'] = \
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:322: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an err

 
Saída: 

Umidade_do_solo: 0.0 %Vol 
Temperatura_do_solo: -35.0 °C 
pH_do_solo: 0.0 pH 
Nivel_de_nutrientes: 0.0 ppm 
Radiacao_solar: 0.0 W/m² 
Umidade_relativa_do_ar: 0.0 % 
Temperatura_ambiente: -55.0 °C 
Vazao: 0.0 L/min 



In [ ]:
# URL do Node-RED
url = "http://127.0.0.1:1880/*"

In [319]:
sensores.send_data_to_NodeRed(url, verbose=False)

Dados enviados com sucesso!


In [320]:
sensores.send_data_to_NodeRed(url, verbose=True)

Dados enviados com sucesso!


Sensores carregados:   0%|          | 0/8 [00:00<?, ?sensores/s]

Sensores carregados: 100%|██████████| 8/8 [00:01<00:00,  4.87sensores/s]

 
Saída: 

Umidade_do_solo: 0.0 %Vol 
Temperatura_do_solo: -35.0 °C 
pH_do_solo: 0.0 pH 
Nivel_de_nutrientes: 0.0 ppm 
Radiacao_solar: 0.0 W/m² 
Umidade_relativa_do_ar: 0.0 % 
Temperatura_ambiente: -55.0 °C 
Vazao: 0.0 L/min 



In [321]:
sensores.send_data_to_NodeRed_from_sensor_data(sensor_data=sensor_data, base_decimal=1023, url=url, verbose=False)

Sensores carregados:   0%|          | 0/8 [00:00<?, ?sensores/s]C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:306: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '%Vol' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_lido_um'] = \
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:313: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_enviado_um'] = \
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:322: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an err

Dados enviados com sucesso!


In [322]:
sensores.send_data_to_NodeRed_from_sensor_data(sensor_data=sensor_data, base_decimal=1023, url=url, verbose=True)

Sensores carregados:   0%|          | 0/8 [00:00<?, ?sensores/s]

C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:306: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '%Vol' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_lido_um'] = \
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:313: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_enviado_um'] = \
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:322: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatib

Dados enviados com sucesso!


Sensores carregados: 100%|██████████| 8/8 [00:01<00:00,  4.87sensores/s]

 
Saída: 

Umidade_do_solo: 0.0 %Vol 
Temperatura_do_solo: -35.0 °C 
pH_do_solo: 0.0 pH 
Nivel_de_nutrientes: 0.0 ppm 
Radiacao_solar: 0.0 W/m² 
Umidade_relativa_do_ar: 0.0 % 
Temperatura_ambiente: -55.0 °C 
Vazao: 0.0 L/min 



In [323]:
umidade_do_solo= [0, 10, 20, 30, 40, 50, 60, 70, 80 , 90, 100]
temperatura_do_solo= [-35, -15, -10, 0 , 10, 15 , 20, 30, 35, 40, 50]
ph_do_solo=[0, 1, 2, 3, 6, 7, 8, 9, 10, 12, 14]
nivel_de_nutrientes= [0, 10, 20, 30, 40, 50, 60, 70, 80 , 90, 100]
radiacao_solar = [0, 100, 200, 300, 400, 500, 600, 700, 800 , 900, 3000]
umidade_relativa_do_ar= [0, 10, 20, 30, 40, 50, 60, 70, 80 , 90, 100]
temperatura_ambiente=[-55,-25, 0, 5, 15, 25, 50, 75, 100, 125,  150]
vazao= [0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 1200]

print(len(umidade_do_solo))
print(len(temperatura_do_solo))
print(len(ph_do_solo))
print(len(nivel_de_nutrientes))
print(len(radiacao_solar))
print(len(umidade_relativa_do_ar))
print(len(temperatura_ambiente))
print(len(vazao))

11
11
11
11
11
11
11
11


In [325]:
total_tarefas= len(vazao)
# Cria a barra de progresso
print("Iniciando...")
time.sleep(2)  # Pausa a execução por 2 segundos
with tqdm(total=total_tarefas, desc="Dados enviados", unit="dados") as pbar:
  # Loop para preenchimento
  for us, ts, ph, nn, rs,ur, ta, va in zip(umidade_do_solo, temperatura_do_solo, ph_do_solo, nivel_de_nutrientes, radiacao_solar,umidade_relativa_do_ar,temperatura_ambiente,vazao ):
    sensor_data= {
                    'Umidade_do_solo': us,
                    'Temperatura_do_solo': ts,
                    'pH_do_solo': ph,
                    'Nivel_de_nutrientes': nn,
                    'Radiacao_solar': rs,
                    'Umidade_relativa_do_ar': ur,
                    'Temperatura_ambiente': ta,
                    'Vazao': va
                  }
    
    sensores.send_data_to_NodeRed_from_sensor_data(sensor_data=sensor_data, base_decimal=1023, url=url, verbose=True)
    time.sleep(0.2)  # Simula uma operação
    pbar.update(1)  # Atualiza a barra com cada tarefa concluída

Iniciando...


Dados enviados:   0%|          | 0/11 [00:00<?, ?dados/s]C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:306: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '%Vol' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_lido_um'] = \
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:313: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_enviado_um'] = \
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:322: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a

Dados enviados com sucesso!


Sensores carregados: 100%|██████████| 8/8 [00:01<00:00,  4.74sensores/s]


 
Saída: 

Umidade_do_solo: 0.0 %Vol 
Temperatura_do_solo: -35.0 °C 
pH_do_solo: 0.0 pH 
Nivel_de_nutrientes: 0.0 ppm 
Radiacao_solar: 0.0 W/m² 
Umidade_relativa_do_ar: 0.0 % 
Temperatura_ambiente: -55.0 °C 
Vazao: 0.0 L/min 



...

Dados enviados:  91%|█████████ | 10/11 [00:39<00:03,  3.99s/dados]C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:306: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '%Vol' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_lido_um'] = \
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:313: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'V' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df_sensores_leitura.loc[self.df_sensores_leitura['sensor'] == sensor, 'Valor_enviado_um'] = \
C:\Users\igbar\AppData\Local\Temp\ipykernel_18768\976975513.py:322: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an e

Dados enviados com sucesso!


Sensores carregados: 100%|██████████| 8/8 [00:01<00:00,  4.76sensores/s]


 
Saída: 

Umidade_do_solo: 100.0 %Vol 
Temperatura_do_solo: 50.0 °C 
pH_do_solo: 14.0 pH 
Nivel_de_nutrientes: 100.0 ppm 
Radiacao_solar: 3000.0 W/m² 
Umidade_relativa_do_ar: 100.0 % 
Temperatura_ambiente: 150.0 °C 
Vazao: 1200.0 L/min 



Dados enviados: 100%|██████████| 11/11 [00:43<00:00,  3.99s/dados]
